In [1]:
%run -i ../header.py
from pylatex import Tabular, MultiColumn, NoEscape, LongTable

In [2]:
data_pack = {
      "b": ["b-trans.deng2017-"+num for num in ("III", "IV", "V", "VI")]
    , "c": ["c-trans.deng2016-"+num for num in ("V", "VI", "VII", "VIII", "IX")]
}

# Export

In [3]:
def getPDGTrans(name):
    # return widths in [KeV]
    pdg_trans = pd.read_csv("../data/{}-trans.PDG.csv".format(name), header=0, index_col=[0,1], comment="#")["frac_width"]
    pdg_trans_err = pd.read_csv("../data/{}-trans.PDG.err.csv".format(name), header=0, index_col=[0,1], comment="#")["frac_width"]
    
    pdg_tot = pd.read_csv("../data/{}-widths.csv".format(name), header=0, index_col=0, comment="#")["PDG"]
    pdg_tot_spec = readSpec("../data/{}-widths.csv".format(name))
    pdg_tot_err = pd.read_csv("../data/{}-widths.err.csv".format(name), header=0, index_col=0, comment="#")["PDG"].abs()
    
    pdg_err = pdg_trans_err.copy()
    
    for instate in pdg_trans.index.levels[0]:
        pdg_err.loc[(instate, slice(None))] = (  pdg_trans_err[(instate, slice(None))].values*pdg_tot[instate]\
                                               + pdg_trans[(instate, slice(None))].values*pdg_tot_err[instate]\
                                              )*dimTrans(pdg_tot_spec["PDG"]["tags"][0], "KeV") # pdg[MeV] -> [KeV]
        pdg_trans.loc[(instate, slice(None))] = pdg_trans[(instate, slice(None))].values*pdg_tot[instate]*dimTrans(pdg_tot_spec["PDG"]["tags"][0], "KeV") # pdg[MeV] -> [KeV]
    
    pdg_trans.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in pdg_trans.index.values], name=["In", "Out"])
    pdg_trans.rename("PDG", inplace=True)
    
    pdg_err.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in pdg_trans_err.index.values], name=["In", "Out"])
    pdg_err = pdg_err.reindex(pdg_trans.index)
    
    return pdg_trans, pdg_err

In [4]:
def getDengTrans(prefix, cfgname):
    # return trans widths of Deng in KeV
    prefixToCol = {"scr": "SP", "lin": "LP"}
    
    col = prefixToCol[prefix]
    spec = readSpec("../data/{}.csv".format(cfgname))
    deng = pd.read_csv("../data/{}.csv".format(cfgname), header=0, index_col=[0,1], comment="#")
    for k, v in spec.items():
        deng[k] *= dimTrans(v["tags"][0], "KeV")
    deng.rename(columns={col: "Deng"}, inplace=True)
    deng.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in deng.index.values], name=["In", "Out"])
    return deng

In [5]:
def getRs(particle, prefix, cfgname, suffix=""):
    # return my widths in KeV
    data = pd.read_csv("output/{}.{}/widths{}".format(prefix, cfgname, suffix), header=0, index_col = [0, 1], comment="#")
    data *= dimTrans("GeV", "KeV")
    data = data.loc[(data != 0).any(axis=1), :]
    data["in0"] /= data["width"]
    data["in2"] /= data["width"]
    data["out0"] /= data["width"]
    data["out2"] /= data["width"]
    data.index.names = ["In", "Out"]
    
    data_sub = data.query("In == '{0}'".format(particle))\
        .rename(columns={"out0": "$r^{(0)}$", "out2": "$r^{(2)}$"})\
        .drop(columns=["in0", "in2"])
    data_above = data.query("Out == '{0}'".format(particle))\
        .rename(columns={"in0": "$r^{(0)}$", "in2": "$r^{(2)}$"})\
        .drop(columns=["out0", "out2"])
    
    data = pd.concat([data_sub, data_above])
    data.columns = ["$\Gamma$", "$r^{(0)}$", "$r^{(2)}$"]
    
    data.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in data.index.values], name=["In", "Out"])
    return data

In [6]:
def reportRs(pname, prefix, cfgname, suffixes=["", "E1"]): # allowed suffixes: "", "E1", "ELW"
    stack = []
    deng_stack = []
    parts = [getRs(pname, prefix, cfgname, suffix)\
                                .add_prefix("" if suffix == "" else suffix+"-")\
                            for suffix in suffixes]
    parts = [part for part in parts if not part.empty]
    if len(parts) > 0:
        stack.append(pd.concat(parts, axis=1))
    
    for dataf in data_pack[cfgname.split("-")[0]]:
        deng_stack.append(getDengTrans(prefix, dataf)["Deng"])
    
    stack = pd.concat(stack, sort=False)
    deng_stack = pd.concat(deng_stack, axis=0, sort=False).reindex(stack.index.values)
    pdg, _ = getPDGTrans(cfgname.split("-")[0])
    pdg = pdg.reindex(stack.index.values)
    
    res = pd.concat([pdg, deng_stack, stack], sort=False, axis=1)
    res = dfsort(res, lambda a,b: cmpMultyStatesByCode(prefix, cfgname
                                                      ,[name2code(i[1:-1]) for i in a]
                                                      ,[name2code(i[1:-1]) for i in b]))
    return res

## Widths

### $\psi(1S)$ and $\Upsilon(1S)$

In [21]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1S", "scr", "c-scr", suffixes=suff)
    display(res)
    with open("export/width{}.psi_1S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.44,2.79,1,0,2.79,1,0
$\chi_{c0}(1P)$,$\psi(1S)$,151,179,196,1,0,160,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,288,319,358,0.436,0,312,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,374,292,333,0.056,0.694,386,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.29,2.91,1,0,2.91,1,0
$\chi_{c0}(2P)$,$\psi(1S)$,nan,2.3,3.89,1,0,2.52,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,88,95.7,0.391,0,74.9,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,117,0.0287,0.764,149,0.1,0.6


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.44,2.79,1,0,nan,nan,nan
$\chi_{c0}(1P)$,$\psi(1S)$,151,179,160,1,0,183,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,288,319,312,0.5,0,382,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,374,292,386,0.1,0.6,490,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.29,2.91,1,0,nan,nan,nan
$\chi_{c0}(2P)$,$\psi(1S)$,nan,2.3,2.52,1,0,3.3,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,88,74.9,0.5,0,51.9,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,149,0.1,0.6,132,0.1,0.6


In [22]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1S", "lin", "c-lin", suffixes=suff)
    display(res)
    with open("export/width{}.psi_1S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.39,2.53,1,0,2.53,1,0
$\chi_{c0}(1P)$,$\psi(1S)$,151,172,187,1,0,154,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,288,306,344,0.439,0,302,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,374,284,325,0.058,0.69,373,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.64,3.04,1,0,3.04,1,0
$\chi_{c0}(2P)$,$\psi(1S)$,nan,6.1,5.24,1,0,3.45,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,81,86.9,0.396,0,68.9,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,116,0.0315,0.756,147,0.1,0.6


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.39,2.53,1,0,nan,nan,nan
$\chi_{c0}(1P)$,$\psi(1S)$,151,172,154,1,0,177,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,288,306,302,0.5,0,367,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,374,284,373,0.1,0.6,471,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.64,3.04,1,0,nan,nan,nan
$\chi_{c0}(2P)$,$\psi(1S)$,nan,6.1,3.45,1,0,1.66,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,81,68.9,0.5,0,46.4,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,147,0.1,0.6,129,0.1,0.6


In [23]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_1S", "scr", "b-scr", suffixes=suff)
    display(res)
    with open("export/width{}.yps_1S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,0.01,0.0107,1,0,0.0107,1,0
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,27.5,27.3,1,0,25.2,1,0
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,31.9,31.6,0.478,0,30.2,0.5,0
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,31.8,31.5,0.0862,0.628,32.9,0.1,0.6
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,5.54,5.59,1,0,4.81,1,0
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.8,11.6,0.461,0,10.7,0.5,0
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,12.5,13.4,0.0759,0.65,14.5,0.1,0.6
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.87,1.86,1,0,1.53,1,0
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,6.41,6.16,0.45,0,5.55,0.5,0


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,0.01,0.0107,1,0,nan,nan,nan
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,27.5,25.2,1,0,26.9,1,0
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,31.9,30.2,0.5,0,32.5,0.5,0
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,31.8,32.9,0.1,0.6,35.7,0.1,0.6
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,5.54,4.81,1,0,3.97,1,0
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.8,10.7,0.5,0,10,0.5,0
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,12.5,14.5,0.1,0.6,14.1,0.1,0.6
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.87,1.53,1,0,1.01,1,0
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,6.41,5.55,0.5,0,4.83,0.5,0


### $\psi(2S)$ and $\Upsilon(2S)$

In [24]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_2S", "scr", "c-scr", suffixes=suff)
    display(res)
    with open("export/width{}.psi_2S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$      1   7.8      10.6          1          0   
                $\chi_{c0}(1P)$  28.8    22      23.6          1          0   
                $\chi_{c1}(1P)$  28.7    45      47.3      0.531          0   
                $\chi_{c2}(1P)$    28    46        47      0.114      0.573   
                $\eta_{c}(2S)$  0.206  0.19     0.233          1          0   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    99       131          1          0   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   155       110      0.469          0   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   150       160     0.0755      0.651   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   9.1      10.8          1          0   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    74      83.2      0.423          0   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    76      85.4     0.0501      0.708   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi(2S)$      $\eta_{c}(1S)$          10.6             1             0  
                $\chi_{c0}(1P)$         28.7             1             0  
                $\chi_{c1}(1P)$         50.3           0.5             0  
                $\chi_{c2}(1P)$         44.9           0.1           0.6  
                $\eta_{c}(2S)$         0.233             1             0  
$\chi_{c0}(2P)$ $\psi(2S)$               116             1             0  
$\chi_{c1}(2P)$ $\psi(2S)$               104           0.5             0  
$\chi_{c2}(2P)$ $\psi(2S)$               174           0.1           0.6  
$\chi_{c0}(3P)$ $\psi(2S)$              8.09             1             0  
$\chi_{c1}(3P)$ $\psi(2S)$              70.3           0.5             0  
$\chi_{c2}(3P)$ $\psi(2S)$               101           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                      
$\psi(2S)$      $\eta_{c}(1S)$      1   7.8         10.6             1   
                $\chi_{c0}(1P)$  28.8    22         28.7             1   
                $\chi_{c1}(1P)$  28.7    45         50.3           0.5   
                $\chi_{c2}(1P)$    28    46         44.9           0.1   
                $\eta_{c}(2S)$  0.206  0.19        0.233             1   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    99          116             1   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   155          104           0.5   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   150          174           0.1   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   9.1         8.09             1   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    74         70.3           0.5   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    76          101           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi(2S)$      $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0          33.5              1   
                $\chi_{c1}(1P)$             0          54.7            0.5   
                $\chi_{c2}(1P)$           0.6          47.5            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi(2S)$                  0           128              1   
$\chi_{c1}(2P)$ $\psi(2S)$                  0           116            0.5   
$\chi_{c2}(2P)$ $\psi(2S)$                0.6           207            0.1   
$\chi_{c0}(3P)$ $\psi(2S)$                  0        0.0396              1   
$\chi_{c1}(3P)$ $\psi(2S)$                  0          40.1            0.5   
$\chi_{c2}(3P)$ $\psi(2S)$                0.6          75.1            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi(2S)$      $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi(2S)$                   0  
$\chi_{c1}(2P)$ $\psi(2S)$                   0  
$\chi_{c2}(2P)$ $\psi(2S)$                 0.6  
$\chi_{c0}(3P)$ $\psi(2S)$                   0  
$\chi_{c1}(3P)$ $\psi(2S)$                   0  
$\chi_{c2}(3P)$ $\psi(2S)$                 0.6

In [25]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_2S", "lin", "c-lin", suffixes=suff)
    display(res)
    with open("export/width{}.psi_2S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$      1  8.08      9.98          1          0   
                $\chi_{c0}(1P)$  28.8    22      24.4          1          0   
                $\chi_{c1}(1P)$  28.7    42      44.4       0.53          0   
                $\chi_{c2}(1P)$    28    38      43.9      0.113      0.575   
                $\eta_{c}(2S)$  0.206  0.19     0.211          1          0   
$\chi_{c0}(2P)$ $\psi(2S)$        nan   121       113          1          0   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   139      97.8       0.47          0   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   135       142     0.0767      0.648   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    17      17.8          1          0   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    94       107      0.415          0   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    97       114     0.0447      0.722   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi(2S)$      $\eta_{c}(1S)$          9.98             1             0  
                $\chi_{c0}(1P)$         29.3             1             0  
                $\chi_{c1}(1P)$           47           0.5             0  
                $\chi_{c2}(1P)$           42           0.1           0.6  
                $\eta_{c}(2S)$         0.211             1             0  
$\chi_{c0}(2P)$ $\psi(2S)$               101             1             0  
$\chi_{c1}(2P)$ $\psi(2S)$                92           0.5             0  
$\chi_{c2}(2P)$ $\psi(2S)$               154           0.1           0.6  
$\chi_{c0}(3P)$ $\psi(2S)$                13             1             0  
$\chi_{c1}(3P)$ $\psi(2S)$                89           0.5             0  
$\chi_{c2}(3P)$ $\psi(2S)$               137           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                      
$\psi(2S)$      $\eta_{c}(1S)$      1  8.08         9.98             1   
                $\chi_{c0}(1P)$  28.8    22         29.3             1   
                $\chi_{c1}(1P)$  28.7    42           47           0.5   
                $\chi_{c2}(1P)$    28    38           42           0.1   
                $\eta_{c}(2S)$  0.206  0.19        0.211             1   
$\chi_{c0}(2P)$ $\psi(2S)$        nan   121          101             1   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   139           92           0.5   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   135          154           0.1   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    17           13             1   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    94           89           0.5   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    97          137           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi(2S)$      $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0          34.1              1   
                $\chi_{c1}(1P)$             0            51            0.5   
                $\chi_{c2}(1P)$           0.6          44.4            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi(2S)$                  0           111              1   
$\chi_{c1}(2P)$ $\psi(2S)$                  0           102            0.5   
$\chi_{c2}(2P)$ $\psi(2S)$                0.6           181            0.1   
$\chi_{c0}(3P)$ $\psi(2S)$                  0         0.466              1   
$\chi_{c1}(3P)$ $\psi(2S)$                  0          37.5            0.5   
$\chi_{c2}(3P)$ $\psi(2S)$                0.6          86.7            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi(2S)$      $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi(2S)$                   0  
$\chi_{c1}(2P)$ $\psi(2S)$                   0  
$\chi_{c2}(2P)$ $\psi(2S)$                 0.6  
$\chi_{c0}(3P)$ $\psi(2S)$                   0  
$\chi_{c1}(3P)$ $\psi(2S)$                   0  
$\chi_{c2}(3P)$ $\psi(2S)$                 0.6

In [26]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_2S", "scr", "b-scr", suffixes=suff)
    display(res)
    with open("export/width{}.yps_2S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                            
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 0.066    0.0737          1          0   
                $\chi_{b0}(1P)$   1.22  1.09      1.09          1          0   
                $\chi_{b1}(1P)$   2.21  2.17      2.19      0.507          0   
                $\chi_{b2}(1P)$   2.29  2.62      2.61      0.103      0.593   
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan  14.4      14.1          1          0   
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan  15.3      15.6      0.488          0   
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan  15.3      15.5     0.0924      0.615   
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan  2.55      2.52          1          0   
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan  5.63      5.31      0.476          0   
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan  6.72      6.46     0.0846      0.632   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\Upsilon(2S)$  $\eta_{b}(1S)$        0.0737             1             0  
                $\chi_{b0}(1P)$         1.13             1             0  
                $\chi_{b1}(1P)$         2.22           0.5             0  
                $\chi_{b2}(1P)$         2.58           0.1           0.6  
$\chi_{b0}(2P)$ $\Upsilon(2S)$          13.5             1             0  
$\chi_{b1}(2P)$ $\Upsilon(2S)$          15.3           0.5             0  
$\chi_{b2}(2P)$ $\Upsilon(2S)$          15.9           0.1           0.6  
$\chi_{b0}(3P)$ $\Upsilon(2S)$          2.29             1             0  
$\chi_{b1}(3P)$ $\Upsilon(2S)$          5.05           0.5             0  
$\chi_{b2}(3P)$ $\Upsilon(2S)$           6.8           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                       
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 0.066       0.0737             1   
                $\chi_{b0}(1P)$   1.22  1.09         1.13             1   
                $\chi_{b1}(1P)$   2.21  2.17         2.22           0.5   
                $\chi_{b2}(1P)$   2.29  2.62         2.58           0.1   
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan  14.4         13.5             1   
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan  15.3         15.3           0.5   
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan  15.3         15.9           0.1   
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan  2.55         2.29             1   
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan  5.63         5.05           0.5   
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan  6.72          6.8           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\Upsilon(2S)$  $\eta_{b}(1S)$              0           nan            nan   
                $\chi_{b0}(1P)$             0          1.16              1   
                $\chi_{b1}(1P)$             0          2.26            0.5   
                $\chi_{b2}(1P)$           0.6          2.62            0.1   
$\chi_{b0}(2P)$ $\Upsilon(2S)$              0          14.2              1   
$\chi_{b1}(2P)$ $\Upsilon(2S)$              0          16.2            0.5   
$\chi_{b2}(2P)$ $\Upsilon(2S)$            0.6          16.9            0.1   
$\chi_{b0}(3P)$ $\Upsilon(2S)$              0          1.62              1   
$\chi_{b1}(3P)$ $\Upsilon(2S)$              0          4.23            0.5   
$\chi_{b2}(3P)$ $\Upsilon(2S)$            0.6          5.98            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\Upsilon(2S)$  $\eta_{b}(1S)$             nan  
                $\chi_{b0}(1P)$              0  
                $\chi_{b1}(1P)$              0  
                $\chi_{b2}(1P)$            0.6  
$\chi_{b0}(2P)$ $\Upsilon(2S)$               0  
$\chi_{b1}(2P)$ $\Upsilon(2S)$               0  
$\chi_{b2}(2P)$ $\Upsilon(2S)$             0.6  
$\chi_{b0}(3P)$ $\Upsilon(2S)$               0  
$\chi_{b1}(3P)$ $\Upsilon(2S)$               0  
$\chi_{b2}(3P)$ $\Upsilon(2S)$             0.6

### $\psi(3S)$ and $\Upsilon(3S)$

In [27]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_3S", "scr", "c-scr", suffixes=suff)
    display(res)
    with open("export/width{}.psi_3S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.76      9.95          1          0   
           $\chi_{c0}(1P)$  nan   6.7      7.85          1          0   
           $\chi_{c1}(1P)$  nan   6.7      7.47      0.595          0   
           $\chi_{c2}(1P)$  nan   2.5      2.62      0.148      0.509   
           $\eta_{c}(2S)$   nan  1.78       2.2          1          0   
           $\chi_{c0}(2P)$  nan    27      22.9          1          0   
           $\chi_{c1}(2P)$  nan    67       113       0.53          0   
           $\chi_{c2}(2P)$  nan    82      82.4      0.112      0.577   
           $\eta_{c}(3S)$   nan 0.088     0.087          1          0   

                            E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In         Out                                                       
$\psi(3S)$ $\eta_{c}(1S)$          9.95             1             0  
           $\chi_{c0}(1P)$         12.4             1             0  
           $\chi_{c1}(1P)$         8.89           0.5             0  
           $\chi_{c2}(1P)$         2.23           0.1           0.6  
           $\eta_{c}(2S)$           2.2             1             0  
           $\chi_{c0}(2P)$           26             1             0  
           $\chi_{c1}(2P)$          120           0.5             0  
           $\chi_{c2}(2P)$         79.2           0.1           0.6  
           $\eta_{c}(3S)$         0.087             1             0

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In         Out                                                     
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.76         9.95             1   
           $\chi_{c0}(1P)$  nan   6.7         12.4             1   
           $\chi_{c1}(1P)$  nan   6.7         8.89           0.5   
           $\chi_{c2}(1P)$  nan   2.5         2.23           0.1   
           $\eta_{c}(2S)$   nan  1.78          2.2             1   
           $\chi_{c0}(2P)$  nan    27           26             1   
           $\chi_{c1}(2P)$  nan    67          120           0.5   
           $\chi_{c2}(2P)$  nan    82         79.2           0.1   
           $\eta_{c}(3S)$   nan 0.088        0.087             1   

                            E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$              0           nan            nan   
           $\chi_{c0}(1P)$             0          8.15              1   
           $\chi_{c1}(1P)$             0         0.753            0.5   
           $\chi_{c2}(1P)$           0.6             2            0.1   
           $\eta_{c}(2S)$              0           nan            nan   
           $\chi_{c0}(2P)$             0          29.8              1   
           $\chi_{c1}(2P)$             0           135            0.5   
           $\chi_{c2}(2P)$           0.6          84.4            0.1   
           $\eta_{c}(3S)$              0           nan            nan   

                            ELW-$r^{(2)}$  
In         Out                             
$\psi(3S)$ $\eta_{c}(1S)$             nan  
           $\chi_{c0}(1P)$              0  
           $\chi_{c1}(1P)$              0  
           $\chi_{c2}(1P)$            0.6  
           $\eta_{c}(2S)$             nan  
           $\chi_{c0}(2P)$              0  
           $\chi_{c1}(2P)$              0  
           $\chi_{c2}(2P)$            0.6  
           $\eta_{c}(3S)$             nan

In [28]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_3S", "lin", "c-lin", suffixes=suff)
    display(res)
    with open("export/width{}.psi_3S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.66      9.06          1          0   
           $\chi_{c0}(1P)$  nan   5.9      6.86          1          0   
           $\chi_{c1}(1P)$  nan     4      4.43       0.59          0   
           $\chi_{c2}(1P)$  nan  0.25     0.176      0.146      0.513   
           $\eta_{c}(2S)$   nan  1.65      1.85          1          0   
           $\chi_{c0}(2P)$  nan    19      20.6          1          0   
           $\chi_{c1}(2P)$  nan    55      92.3      0.528          0   
           $\chi_{c2}(2P)$  nan    67      66.7      0.111      0.578   

                            E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In         Out                                                       
$\psi(3S)$ $\eta_{c}(1S)$          9.06             1             0  
           $\chi_{c0}(1P)$         10.6             1             0  
           $\chi_{c1}(1P)$         5.23           0.5             0  
           $\chi_{c2}(1P)$        0.151           0.1           0.6  
           $\eta_{c}(2S)$          1.85             1             0  
           $\chi_{c0}(2P)$         23.3             1             0  
           $\chi_{c1}(2P)$         97.6           0.5             0  
           $\chi_{c2}(2P)$         64.3           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In         Out                                                     
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.66         9.06             1   
           $\chi_{c0}(1P)$  nan   5.9         10.6             1   
           $\chi_{c1}(1P)$  nan     4         5.23           0.5   
           $\chi_{c2}(1P)$  nan  0.25        0.151           0.1   
           $\eta_{c}(2S)$   nan  1.65         1.85             1   
           $\chi_{c0}(2P)$  nan    19         23.3             1   
           $\chi_{c1}(2P)$  nan    55         97.6           0.5   
           $\chi_{c2}(2P)$  nan    67         64.3           0.1   

                            E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$              0           nan            nan   
           $\chi_{c0}(1P)$             0          6.21              1   
           $\chi_{c1}(1P)$             0        0.0879            0.5   
           $\chi_{c2}(1P)$           0.6          5.56            0.1   
           $\eta_{c}(2S)$              0           nan            nan   
           $\chi_{c0}(2P)$             0          26.3              1   
           $\chi_{c1}(2P)$             0           109            0.5   
           $\chi_{c2}(2P)$           0.6            68            0.1   

                            ELW-$r^{(2)}$  
In         Out                             
$\psi(3S)$ $\eta_{c}(1S)$             nan  
           $\chi_{c0}(1P)$              0  
           $\chi_{c1}(1P)$              0  
           $\chi_{c2}(1P)$            0.6  
           $\eta_{c}(2S)$             nan  
           $\chi_{c0}(2P)$              0  
           $\chi_{c1}(2P)$              0  
           $\chi_{c2}(2P)$            0.6

In [29]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_3S", "scr", "b-scr", suffixes=suff)
    display(res)
    with open("export/width{}.yps_3S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG   Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                             
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104  0.071    0.0795          1          0   
                $\chi_{b0}(1P)$ 0.0549  0.097    0.0903          1          0   
                $\chi_{b1}(1P)$ 0.0183 0.0005  8.08e-05      0.524          0   
                $\chi_{b2}(1P)$  0.201   0.14      0.16      0.114      0.573   
                $\eta_{b}(2S)$     nan  0.011    0.0122          1          0   
                $\chi_{b0}(2P)$    1.2   1.21      1.23          1          0   
                $\chi_{b1}(2P)$   2.56   2.61      2.58      0.505          0   
                $\chi_{b2}(2P)$   2.66   3.16      3.21      0.103      0.595   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan   7.95      7.93          1          0   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan   10.3      9.55      0.492          0   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan   10.8      10.6     0.0946      0.611   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\Upsilon(3S)$  $\eta_{b}(1S)$        0.0795             1             0  
                $\chi_{b0}(1P)$          0.1             1             0  
                $\chi_{b1}(1P)$     8.47e-05           0.5             0  
                $\chi_{b2}(1P)$        0.153           0.1           0.6  
                $\eta_{b}(2S)$        0.0122             1             0  
                $\chi_{b0}(2P)$         1.27             1             0  
                $\chi_{b1}(2P)$          2.6           0.5             0  
                $\chi_{b2}(2P)$         3.18           0.1           0.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$          7.71             1             0  
$\chi_{b1}(3P)$ $\Upsilon(3S)$           9.4           0.5             0  
$\chi_{b2}(3P)$ $\Upsilon(3S)$          10.8           0.1           0.6

['E1', 'ELW']


PDG   Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                        
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104  0.071       0.0795             1   
                $\chi_{b0}(1P)$ 0.0549  0.097          0.1             1   
                $\chi_{b1}(1P)$ 0.0183 0.0005     8.47e-05           0.5   
                $\chi_{b2}(1P)$  0.201   0.14        0.153           0.1   
                $\eta_{b}(2S)$     nan  0.011       0.0122             1   
                $\chi_{b0}(2P)$    1.2   1.21         1.27             1   
                $\chi_{b1}(2P)$   2.56   2.61          2.6           0.5   
                $\chi_{b2}(2P)$   2.66   3.16         3.18           0.1   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan   7.95         7.71             1   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan   10.3          9.4           0.5   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan   10.8         10.8           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\Upsilon(3S)$  $\eta_{b}(1S)$              0           nan            nan   
                $\chi_{b0}(1P)$             0        0.0436              1   
                $\chi_{b1}(1P)$             0        0.0413            0.5   
                $\chi_{b2}(1P)$           0.6         0.455            0.1   
                $\eta_{b}(2S)$              0           nan            nan   
                $\chi_{b0}(2P)$             0           1.3              1   
                $\chi_{b1}(2P)$             0          2.65            0.5   
                $\chi_{b2}(2P)$           0.6          3.23            0.1   
$\chi_{b0}(3P)$ $\Upsilon(3S)$              0          7.98              1   
$\chi_{b1}(3P)$ $\Upsilon(3S)$              0          9.84            0.5   
$\chi_{b2}(3P)$ $\Upsilon(3S)$            0.6          11.5            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\Upsilon(3S)$  $\eta_{b}(1S)$             nan  
                $\chi_{b0}(1P)$              0  
                $\chi_{b1}(1P)$              0  
                $\chi_{b2}(1P)$            0.6  
                $\eta_{b}(2S)$             nan  
                $\chi_{b0}(2P)$              0  
                $\chi_{b1}(2P)$              0  
                $\chi_{b2}(2P)$            0.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$               0  
$\chi_{b1}(3P)$ $\Upsilon(3S)$               0  
$\chi_{b2}(3P)$ $\Upsilon(3S)$             0.6

### $\psi_1(1D)$ which is $\psi(3770)$

In [30]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1_1D", "scr", "c-scr", suffixes=suff)
    display(res)
    with open("export/width{}.psi_1_1D.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan     0.386          1          0   
                $\chi_{c0}(1P)$  188   261       291          1          0   
                $\chi_{c1}(1P)$ 67.7   135       153      0.467          0   
                $\chi_{c2}(1P)$  nan   8.1      8.04      0.103      0.717   
                $\eta_{c}(2S)$   nan   nan   0.00011          1          0   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    12      20.7          1          0   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   9.8      4.86      0.513          0   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.46     0.522      0.211      0.574   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.39      0.32          1          0   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     2      2.28      0.581          0   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.79      0.93      0.176   0.000393   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$         0.386             1             0  
                $\chi_{c0}(1P)$          259             1             0  
                $\chi_{c1}(1P)$          129           0.5             0  
                $\chi_{c2}(1P)$         5.93           0.1           0.6  
                $\eta_{c}(2S)$       0.00011             1             0  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$          21.3             1             0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          5.19           0.5             0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         0.632           0.1           0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$         0.364             1             0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$          3.05           0.5             0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$         0.225           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan        0.386             1   
                $\chi_{c0}(1P)$  188   261          259             1   
                $\chi_{c1}(1P)$ 67.7   135          129           0.5   
                $\chi_{c2}(1P)$  nan   8.1         5.93           0.1   
                $\eta_{c}(2S)$   nan   nan      0.00011             1   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    12         21.3             1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   9.8         5.19           0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.46        0.632           0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.39        0.364             1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     2         3.05           0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.79        0.225           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0           318              1   
                $\chi_{c1}(1P)$             0           148            0.5   
                $\chi_{c2}(1P)$           0.6          6.61            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$              0          22.1              1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$              0          5.42            0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$            0.6         0.692            0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$              0          9.49              1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$              0         0.392            0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$            0.6        0.0791            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$             0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$             0.6

In [31]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1_1D", "lin", "c-lin", suffixes=suff)
    display(res)
    with open("export/width{}.psi_1_1D.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan     0.341          1          0   
                $\chi_{c0}(1P)$  188   272       296          1          0   
                $\chi_{c1}(1P)$ 67.7   138       145      0.468          0   
                $\chi_{c2}(1P)$  nan   7.1       7.5      0.101      0.711   
                $\eta_{c}(2S)$   nan   nan  7.92e-05          1          0   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    20      16.6          1          0   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   7.9      3.87      0.513          0   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.36     0.402        0.2      0.574   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.27      0.16          1          0   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   3.2      3.49      0.598          0   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   1.5      1.76      0.201    0.00477   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$         0.341             1             0  
                $\chi_{c0}(1P)$          264             1             0  
                $\chi_{c1}(1P)$          123           0.5             0  
                $\chi_{c2}(1P)$         5.62           0.1           0.6  
                $\eta_{c}(2S)$      7.92e-05             1             0  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$          17.1             1             0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          4.12           0.5             0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         0.484           0.1           0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$         0.186             1             0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$          4.91           0.5             0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$         0.343           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan        0.341             1   
                $\chi_{c0}(1P)$  188   272          264             1   
                $\chi_{c1}(1P)$ 67.7   138          123           0.5   
                $\chi_{c2}(1P)$  nan   7.1         5.62           0.1   
                $\eta_{c}(2S)$   nan   nan     7.92e-05             1   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    20         17.1             1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   7.9         4.12           0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.36        0.484           0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.27        0.186             1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   3.2         4.91           0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   1.5        0.343           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$              0           nan            nan   
                $\chi_{c0}(1P)$             0           321              1   
                $\chi_{c1}(1P)$             0           140            0.5   
                $\chi_{c2}(1P)$           0.6          6.24            0.1   
                $\eta_{c}(2S)$              0           nan            nan   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$              0          17.7              1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$              0          4.29            0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$            0.6         0.527            0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$              0          9.27              1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$              0         0.345            0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$            0.6        0.0916            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$             0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$             0.6

## Sumrules for the thesis

In [7]:
def getMasses(prefix, cfgname):
    # return masses in MeV
    with open("../quarkEigen/output/{}.{}/config".format(prefix, cfgname), "r") as f:
        mass_cfg = json.load(f, object_pairs_hook=OrderedDict)
    with open("../quarkEigen/output/{}.{}/exclude".format(prefix, cfgname), "r") as f:
        mass_excl = f.read().split("\n")
    mass_data = OrderedDict()
    for particle, pdata in mass_cfg.items():
        if particle not in mass_excl:
            continue
        energ_data = pd.read_csv("../quarkEigen/output/{}.{}/data/{}/minE.dat".format(prefix, cfgname, particle))\
                     .iloc[-1, 1]
        energ = pdata["eq"]["env"]["mC"]*2 + energ_data
        mass_data.update({particle: energ})
    mass_data = pd.DataFrame.from_dict(mass_data, orient="index")
    mass_data.columns = ["E"]
    mass_data["E"] *= dimTrans("GeV", "MeV")
    pdg_spec = readSpec("../data/{}-spectrum.csv".format(cfgname.split("-")[0]))["PDG"]
    pdg = pd.read_csv("../data/{}-spectrum.csv".format(cfgname.split("-")[0]), header=0, index_col=0, comment="#")["PDG"]\
        *dimTrans(pdg_spec["tags"][0], "MeV")
    pdg = pdg.loc[pdg.index.intersection(mass_data.index).values]
    mass_data.loc[~pd.isna(pdg), "E"] = pdg.loc[~pd.isna(pdg)]
    mass_data.index = pd.Index(["${}$".format(code2name(k)) for k in mass_data.index.values], name="States")
    return mass_data

In [8]:
def getPprops(prefix, cfgname):
    with open("../quarkEigen/output/{}.{}/config".format(prefix, cfgname), "r") as f:
        pprops = json.load(f, object_pairs_hook=OrderedDict)
    props = pd.DataFrame(OrderedDict([(particle, values["eq"]) for particle, values in pprops.items()])).T
    props.drop(columns=["env", "E"], inplace=True)
    props.index = pd.Index([r"${}$".format(code2name(particle)) for particle in pprops.keys()], name="States")
    props = props.apply(pd.to_numeric, axis=0)
    return props

In [9]:
def getSumrule(pname, prefix, cfgname, suffix=""):
    msuffix = "" if suffix == "" else suffix+"-" 
    pretty_pname = "${}$".format(code2name(pname))
    
    widths = reportRs(pname, prefix, cfgname, [suffix])
    masses = getMasses(prefix, cfgname)
    pprops = getPprops(prefix, cfgname)
    
    sumrule = []

    abovethr = [p[0] for p in widths.query("Out == r'{}'".format(pretty_pname)).index.values]
    sumrule.append(
        8*sp.pi**2*masses.loc[abovethr, "E"]**3\
        /(masses.loc[abovethr, "E"]**2 - masses.loc[pretty_pname, "E"]**2)**3\
        *pprops.loc[abovethr, "xJ"]
    )
    sumrule[0].index = pd.Index([(v, pretty_pname) for v in sumrule[0].index.values])
    
    belowthr = [p[1] for p in widths.query("In == r'{}'".format(pretty_pname)).index.values]
    sumrule.append(
        8*sp.pi**2*masses.loc[pretty_pname, "E"]**3\
        /(masses.loc[pretty_pname, "E"]**2 - masses.loc[belowthr, "E"]**2)**3\
        *pprops.loc[pretty_pname, "xJ"]
    )
    sumrule[1].index = pd.Index([(pretty_pname, v) for v in sumrule[1].index.values])
    sr = pd.concat(sumrule)*(0.19732697)**2*10**4*10**3
    sr = sr.reindex(widths.index.values)
    
    widths["$SR-PDG$"] = widths["PDG"].values
    if sp.any(pd.isna(widths["PDG"])):
        widths.loc[pd.isna(widths["PDG"]), "$SR-PDG$"] = widths.loc[pd.isna(widths["PDG"]).index.values, msuffix+"$\Gamma$"]
    widths.loc[:, "$SR-PDG$"] *= sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths["$SR-Deng$"] = widths["Deng"]
    if sp.any(pd.isna(widths["Deng"])):
        widths.loc[pd.isna(widths["Deng"]), "$SR-Deng$"] = widths.loc[pd.isna(widths["Deng"]).index.values, msuffix+"$\Gamma$"]
    widths.loc[:, "$SR-Deng$"] *= sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths["$SR-\Gamma$"] = widths[msuffix+"$\Gamma$"]*sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths.drop(columns=[msuffix+'$r^{(2)}$', msuffix+'$r^{(0)}$'], inplace=True)
    widths = widths[[widths.columns[i] for i in [0, 3, 1, 4, 2, 5]]]
    widths = dfsort(widths, lambda a,b: cmpMultyStatesByCode(prefix, cfgname
                                                         ,[name2code(i[1:-1]) for i in a]
                                                         ,[name2code(i[1:-1]) for i in b]))
    
    sigma = (widths["$SR-Deng$"] - widths["$SR-\Gamma$"]).std()
    
    return widths, pd.DataFrame(widths[["$SR-PDG$", "$SR-Deng$", "$SR-\Gamma$"]].sum().rename("$SR$")), sigma

In [10]:
def reportSumrules(pname, prefix, cfgname, suffixes=("", "E1", "ELW"), sigma=0):
    if sigma is not None:
        try:
            sigma = list(sigma)
        except TypeError:
            sigma = [sigma]*len(suffixes)
        sigma = dict(zip(suffixes, sigma))
    for approx in suffixes:
        display("Approximation: "+(approx if len(approx) > 0 else "tot"))
        report, sr, autosigma = getSumrule(pname, prefix, cfgname, approx)
        display(sr)
        sigma[approx] = autosigma if sigma[approx] == 0 else sigma[approx]
        display(sigma[approx])
        display(report)
        
        tab = Tabular("|l|l|c|c|c|c|c|c|")
        
        # Header
        tab.add_hline()
        tab.add_row(["", ""] + [NoEscape(i) for i in report.columns])
        tab.add_row([NoEscape(i) for i in report.index.names] + [""]*6)
        tab.add_hline()
        
        curN = "1"
        tot = report.iloc[0, 1::2].fillna(0)*0
        getN = re.compile("\((\d+)[A-Z]\)\$$")
        for entry in report.index.values:
            # Manage subtotals
            label = entry[0] if entry[0] != "${}$".format(code2name(pname)) else entry[1]
            newN = getN.search(label).groups()[0]
            if newN != curN:
                tab.add_hline()
                tab.add_hline()
                try:
                    tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                            [MultiColumn(2, align="|r|", data=NoEscape("${:.2f} ({:.2f}\sigma)$".format(num, num/sigma[approx]))) for num in tot])
                except:
                    tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                            [MultiColumn(2, align="|r|", data="{:.2f}".format(num)) for num in tot])                
                tab.add_hline()
                tab.add_hline()
                tot = report.loc[entry].fillna(0).values[1::2]
                curN = newN
            else:
                tot += report.loc[entry].fillna(0).values[1::2]
            
            # Regular entry
            tab.add_row([NoEscape(i) for i in list(entry) + ["{:.2f}".format(num) for num in report.loc[entry]]])
            
        # Final subtotal
        tab.add_hline()
        tab.add_hline()
        try:
            tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                    [MultiColumn(2, align="|r|", data=NoEscape("${:.2f}~({:.2f}\sigma)$".format(num, num/sigma[approx]))) for num in tot])
        except:
            tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                    [MultiColumn(2, align="|r|", data="{:.2f}".format(num)) for num in tot])                
        tab.add_hline()
        tab.add_hline()
        
        # Total
        try:
            tab.add_row([MultiColumn(2, align="|c|", data="Total")] +\
                    [MultiColumn(2, align="|r|", data=NoEscape("${:.2f}~({:.2f}\sigma)$".format(num, num/sigma[approx]))) for num in sr.iloc[:, 0]])
        except:
            tab.add_row([MultiColumn(2, align="|c|", data="Total")] +\
                    [MultiColumn(2, align="|r|", data="{:.2f}".format(num)) for num in sr.iloc[:, 0]])                
        tab.add_hline()
        
        # with open("export/sr{}.{}.{}.tex".format(approx, pname, cfgcode), "w") as f:
        #     f.write(sr.to_latex(escape=False, bold_rows=True, column_format="l|c"))
        # with open("export/sr{}-report.{}.{}.tex".format(approx, pname, cfgcode), "w") as f:
        #     f.write(report.to_latex(escape=False, bold_rows=True, column_format="l|l|c|c|c|c|c|c"))
        tab.generate_tex("export/sr{}-report.{}.{}".format(approx, pname, cfgname))

## Bottomonium sumrules

### $\Upsilon(1S)$

In [36]:
reportSumrules("yps_1S", "scr", "b-scr", sigma=[0]*3)

'Approximation: tot'

,$SR$
$SR-PDG$,-0.621
$SR-Deng$,-0.587
$SR-\Gamma$,-0.621


0.021196853751583623

,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,-0.542,0.01,-0.505,0.0107,-0.542
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.76,27.5,-1.77,27.3,-1.76
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.3,31.9,-2.32,31.6,-2.3
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,3.81,31.8,3.85,31.5,3.81
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0523,5.54,-0.0519,5.59,-0.0523
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.138,10.8,-0.129,11.6,-0.138
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.315,12.5,0.295,13.4,0.315
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00758,1.87,-0.00763,1.86,-0.00758
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0321,6.41,-0.0334,6.16,-0.0321


'Approximation: E1'

...

,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\Upsilon(1S)$,$\eta_{b}(1S)$,nan,-0.542,0.01,-0.505,0.0107,-0.542
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.62,27.5,-1.77,25.2,-1.62
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.3,31.9,-2.43,30.2,-2.3
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,3.68,31.8,3.55,32.9,3.68
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.045,5.54,-0.0519,4.81,-0.045
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.138,10.8,-0.139,10.7,-0.138
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.297,12.5,0.256,14.5,0.297
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00623,1.87,-0.00763,1.53,-0.00623
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0321,6.41,-0.0371,5.55,-0.0321


'Approximation: ELW'

,$SR$
$SR-PDG$,-0.0587
$SR-Deng$,-0.553
$SR-\Gamma$,-0.0587


0.14386532113364772

,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.73,27.5,-1.77,26.9,-1.73
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.48,31.9,-2.43,32.5,-2.48
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,3.98,31.8,3.55,35.7,3.98
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0372,5.54,-0.0519,3.97,-0.0372
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.129,10.8,-0.139,10,-0.129
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.288,12.5,0.256,14.1,0.288
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00414,1.87,-0.00763,1.01,-0.00414
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0279,6.41,-0.0371,4.83,-0.0279
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,0.0747,8.17,0.0753,8.1,0.0747


### $\Upsilon(2S)$

In [37]:
reportSumrules("yps_2S", "scr", "b-scr", sigma=[0]*3)

'Approximation: tot'

,$SR$
$SR-PDG$,-1.88
$SR-Deng$,-0.128
$SR-\Gamma$,-0.141


0.08310164033194033

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                         
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 -0.00065 0.066  -0.00344    0.0737   
                $\chi_{b0}(1P)$   1.22    -3.27  1.09     -2.93      1.09   
                $\chi_{b1}(1P)$   2.21    -5.92  2.17     -5.82      2.19   
                $\chi_{b2}(1P)$   2.29     9.58  2.62        11      2.61   
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan    -6.11  14.4     -6.23      14.1   
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan    -7.28  15.3     -7.12      15.6   
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan     10.9  15.3      10.8      15.5   
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan   -0.101  2.55    -0.102      2.52   
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan   -0.267  5.63    -0.284      5.31   
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan    0.567  6.72      0.59      6.46   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\eta_{b}(1S)$     -0.00384  
                $\chi_{b0}(1P)$       -2.93  
                $\chi_{b1}(1P)$       -5.86  
                $\chi_{b2}(1P)$        10.9  
$\chi_{b0}(2P)$ $\Upsilon(2S)$        -6.11  
$\chi_{b1}(2P)$ $\Upsilon(2S)$        -7.28  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         10.9  
$\chi_{b0}(3P)$ $\Upsilon(2S)$       -0.101  
$\chi_{b1}(3P)$ $\Upsilon(2S)$       -0.267  
$\chi_{b2}(3P)$ $\Upsilon(2S)$        0.567

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                            
$\Upsilon(2S)$  $\eta_{b}(1S)$  0.0125 -0.00065 0.066  -0.00344       0.0737   
                $\chi_{b0}(1P)$   1.22    -3.27  1.09     -2.93         1.13   
                $\chi_{b1}(1P)$   2.21    -5.84  2.17     -5.74         2.22   
                $\chi_{b2}(1P)$   2.29     9.79  2.62      11.2         2.58   
$\chi_{b0}(2P)$ $\Upsilon(2S)$     nan    -5.85  14.4     -6.23         13.5   
$\chi_{b1}(2P)$ $\Upsilon(2S)$     nan    -7.28  15.3     -7.29         15.3   
$\chi_{b2}(2P)$ $\Upsilon(2S)$     nan     10.7  15.3      10.3         15.9   
$\chi_{b0}(3P)$ $\Upsilon(2S)$     nan  -0.0918  2.55    -0.102         2.29   
$\chi_{b1}(3P)$ $\Upsilon(2S)$     nan   -0.267  5.63    -0.298         5.05   
$\chi_{b2}(3P)$ $\Upsilon(2S)$     nan    0.545  6.72     0.539          6.8   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\eta_{b}(1S)$     -0.00384  
                $\chi_{b0}(1P)$       -3.03  
                $\chi_{b1}(1P)$       -5.86  
                $\chi_{b2}(1P)$          11  
$\chi_{b0}(2P)$ $\Upsilon(2S)$        -5.85  
$\chi_{b1}(2P)$ $\Upsilon(2S)$        -7.28  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         10.7  
$\chi_{b0}(3P)$ $\Upsilon(2S)$      -0.0918  
$\chi_{b1}(3P)$ $\Upsilon(2S)$       -0.267  
$\chi_{b2}(3P)$ $\Upsilon(2S)$        0.545

'Approximation: ELW'

,$SR$
$SR-PDG$,-1.55
$SR-Deng$,-0.526
$SR-\Gamma$,-0.113


0.42661577948157475

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22    -3.27  1.09     -2.93          1.16   
                $\chi_{b1}(1P)$ 2.21    -5.84  2.17     -5.74          2.26   
                $\chi_{b2}(1P)$ 2.29     9.79  2.62      11.2          2.62   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan    -6.12  14.4     -6.23          14.2   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan     -7.7  15.3     -7.29          16.2   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan     11.4  15.3      10.3          16.9   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan  -0.0648  2.55    -0.102          1.62   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan   -0.224  5.63    -0.298          4.23   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan     0.48  6.72     0.539          5.98   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$       -3.12  
                $\chi_{b1}(1P)$       -5.98  
                $\chi_{b2}(1P)$        11.2  
$\chi_{b0}(2P)$ $\Upsilon(2S)$        -6.12  
$\chi_{b1}(2P)$ $\Upsilon(2S)$         -7.7  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         11.4  
$\chi_{b0}(3P)$ $\Upsilon(2S)$      -0.0648  
$\chi_{b1}(3P)$ $\Upsilon(2S)$       -0.224  
$\chi_{b2}(3P)$ $\Upsilon(2S)$         0.48

### $\Upsilon(3S)$

In [38]:
reportSumrules("yps_3S", "scr", "b-scr")

'Approximation: tot'

,$SR$
$SR-PDG$,-5.08
$SR-Deng$,-1.88
$SR-\Gamma$,-0.553


0.38460166785490635

PDG  $SR-PDG$   Deng $SR-Deng$  $\Gamma$  \
In              Out                                                           
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104 -0.000157  0.071  -0.00108    0.0795   
                $\chi_{b0}(1P)$ 0.0549  -0.00558  0.097  -0.00987    0.0903   
                $\chi_{b1}(1P)$ 0.0183   -0.0012 0.0005 -3.27e-05  8.08e-05   
                $\chi_{b2}(1P)$  0.201    0.0131   0.14    0.0091      0.16   
                $\eta_{b}(2S)$     nan  -0.00327  0.011  -0.00296    0.0122   
                $\chi_{b0}(2P)$    1.2     -7.62   1.21     -7.69      1.23   
                $\chi_{b1}(2P)$   2.56     -15.3   2.61     -15.5      2.58   
                $\chi_{b2}(2P)$   2.66      23.6   3.16        28      3.21   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan     -12.3   7.95     -12.3      7.93   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan     -14.4   10.3     -15.5      9.55   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan      20.9   10.8      21.2      10.6   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(3S)$  $\eta_{b}(1S)$     -0.00121  
                $\chi_{b0}(1P)$    -0.00919  
                $\chi_{b1}(1P)$   -5.28e-06  
                $\chi_{b2}(1P)$      0.0104  
                $\eta_{b}(2S)$     -0.00327  
                $\chi_{b0}(2P)$       -7.84  
                $\chi_{b1}(2P)$       -15.4  
                $\chi_{b2}(2P)$        28.4  
$\chi_{b0}(3P)$ $\Upsilon(3S)$        -12.3  
$\chi_{b1}(3P)$ $\Upsilon(3S)$        -14.4  
$\chi_{b2}(3P)$ $\Upsilon(3S)$         20.9

'Approximation: E1'

...

PDG  $SR-PDG$   Deng $SR-Deng$  \
In              Out                                                 
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104 -0.000157  0.071  -0.00108   
                $\chi_{b0}(1P)$ 0.0549  -0.00558  0.097  -0.00987   
                $\chi_{b1}(1P)$ 0.0183  -0.00114 0.0005 -3.12e-05   
                $\chi_{b2}(1P)$  0.201    0.0142   0.14   0.00991   
                $\eta_{b}(2S)$     nan  -0.00327  0.011  -0.00296   
                $\chi_{b0}(2P)$    1.2     -7.62   1.21     -7.69   
                $\chi_{b1}(2P)$   2.56     -15.1   2.61     -15.4   
                $\chi_{b2}(2P)$   2.66        24   3.16      28.5   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan       -12   7.95     -12.3   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan     -14.4   10.3     -15.8   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan      20.6   10.8      20.5   

                                 E1-$\Gamma$ $SR-\Gamma$  
In              Out                                       
$\Upsilon(3S)$  $\eta_{b}(1S)$        0.0795    -0.00121  
                $\chi_{b0}(1P)$          0.1     -0.0102  
                $\chi_{b1}(1P)$     8.47e-05   -5.28e-06  
                $\chi_{b2}(1P)$        0.153      0.0108  
                $\eta_{b}(2S)$        0.0122    -0.00327  
                $\chi_{b0}(2P)$         1.27       -8.05  
                $\chi_{b1}(2P)$          2.6       -15.4  
                $\chi_{b2}(2P)$         3.18        28.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$          7.71         -12  
$\chi_{b1}(3P)$ $\Upsilon(3S)$           9.4       -14.4  
$\chi_{b2}(3P)$ $\Upsilon(3S)$          10.8        20.6

'Approximation: ELW'

,$SR$
$SR-PDG$,-4.38
$SR-Deng$,-2.17
$SR-\Gamma$,-0.436


0.5617698084348861

PDG $SR-PDG$   Deng $SR-Deng$  \
In              Out                                                
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549 -0.00558  0.097  -0.00987   
                $\chi_{b1}(1P)$ 0.0183 -0.00114 0.0005 -3.12e-05   
                $\chi_{b2}(1P)$  0.201   0.0142   0.14   0.00991   
                $\chi_{b0}(2P)$    1.2    -7.62   1.21     -7.69   
                $\chi_{b1}(2P)$   2.56    -15.1   2.61     -15.4   
                $\chi_{b2}(2P)$   2.66       24   3.16      28.5   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan    -12.4   7.95     -12.3   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan    -15.1   10.3     -15.8   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan     21.8   10.8      20.5   

                                 ELW-$\Gamma$ $SR-\Gamma$  
In              Out                                        
$\Upsilon(3S)$  $\chi_{b0}(1P)$        0.0436    -0.00443  
                $\chi_{b1}(1P)$        0.0413    -0.00257  
                $\chi_{b2}(1P)$         0.455      0.0322  
                $\chi_{b0}(2P)$           1.3       -8.25  
                $\chi_{b1}(2P)$          2.65       -15.6  
                $\chi_{b2}(2P)$          3.23        29.1  
$\chi_{b0}(3P)$ $\Upsilon(3S)$           7.98       -12.4  
$\chi_{b1}(3P)$ $\Upsilon(3S)$           9.84       -15.1  
$\chi_{b2}(3P)$ $\Upsilon(3S)$           11.5        21.8

## Charmonium sumrules

### $\psi(1S)$

In [39]:
reportSumrules("psi_1S", "scr", "c-scr", sigma=6.7)

'Approximation: tot'

,$SR$
$SR-PDG$,1.79
$SR-Deng$,-25.4
$SR-\Gamma$,-27


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.44,-20.6,2.79,-23.6
$\chi_{c0}(1P)$,$\psi(1S)$,151,-20.9,179,-24.7,196,-27.1
$\chi_{c1}(1P)$,$\psi(1S)$,288,-24.5,319,-27.1,358,-30.5
$\chi_{c2}(1P)$,$\psi(1S)$,374,57.9,292,45.2,333,51.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0893,2.29,-0.0702,2.91,-0.0893
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0457,2.3,-0.027,3.89,-0.0457
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.27,88,-1.17,95.7,-1.27
$\chi_{c2}(2P)$,$\psi(1S)$,nan,4.03,93,3.21,117,4.03


'Approximation: E1'

...

,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.44,-20.6,2.79,-23.6
$\chi_{c0}(1P)$,$\psi(1S)$,151,-20.9,179,-24.7,160,-22.1
$\chi_{c1}(1P)$,$\psi(1S)$,288,-28.1,319,-31.1,312,-30.5
$\chi_{c2}(1P)$,$\psi(1S)$,374,45.4,292,35.4,386,46.8
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0893,2.29,-0.0702,2.91,-0.0893
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0295,2.3,-0.027,2.52,-0.0295
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.27,88,-1.5,74.9,-1.27
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.49,93,2.18,149,3.49


'Approximation: ELW'

,$SR$
$SR-PDG$,-1.49
$SR-Deng$,-19.8
$SR-\Gamma$,-1.12


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{c0}(1P)$,$\psi(1S)$,151,-20.9,179,-24.7,183,-25.3
$\chi_{c1}(1P)$,$\psi(1S)$,288,-28.1,319,-31.1,382,-37.3
$\chi_{c2}(1P)$,$\psi(1S)$,374,45.4,292,35.4,490,59.3
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0387,2.3,-0.027,3.3,-0.0387
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-0.883,88,-1.5,51.9,-0.883
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.09,93,2.18,132,3.09


In [40]:
reportSumrules("psi_1S", "lin", "c-lin", sigma=6.7)

'Approximation: tot'

,$SR$
$SR-PDG$,1.04
$SR-Deng$,-24.8
$SR-\Gamma$,-24.4


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.39,-20.2,2.53,-21.3
$\chi_{c0}(1P)$,$\psi(1S)$,151,-20.9,172,-23.8,187,-25.9
$\chi_{c1}(1P)$,$\psi(1S)$,288,-24.7,306,-26.2,344,-29.4
$\chi_{c2}(1P)$,$\psi(1S)$,374,57.3,284,43.5,325,49.7
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.093,2.64,-0.0809,3.04,-0.093
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0614,6.1,-0.0716,5.24,-0.0614
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.17,81,-1.09,86.9,-1.17
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.96,93,3.16,116,3.96


'Approximation: E1'

...

,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.39,-20.2,2.53,-21.3
$\chi_{c0}(1P)$,$\psi(1S)$,151,-20.9,172,-23.8,154,-21.3
$\chi_{c1}(1P)$,$\psi(1S)$,288,-28.1,306,-29.9,302,-29.4
$\chi_{c2}(1P)$,$\psi(1S)$,374,45.4,284,34.4,373,45.2
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.093,2.64,-0.0809,3.04,-0.093
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0405,6.1,-0.0716,3.45,-0.0405
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-1.17,81,-1.38,68.9,-1.17
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.44,93,2.18,147,3.44


'Approximation: ELW'

,$SR$
$SR-PDG$,-1.45
$SR-Deng$,-18.5
$SR-\Gamma$,-0.975


6.7

,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{c0}(1P)$,$\psi(1S)$,151,-20.9,172,-23.8,177,-24.4
$\chi_{c1}(1P)$,$\psi(1S)$,288,-28.1,306,-29.9,367,-35.9
$\chi_{c2}(1P)$,$\psi(1S)$,374,45.4,284,34.4,471,57.1
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0195,6.1,-0.0716,1.66,-0.0195
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-0.789,81,-1.38,46.4,-0.789
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.02,93,2.18,129,3.02


### $\psi(2S)$

In [41]:
reportSumrules("psi_2S", "scr", "c-scr", sigma=16.6)

'Approximation: tot'

,$SR$
$SR-PDG$,-59.4
$SR-Deng$,-53.5
$SR-\Gamma$,-33.6


16.6

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi(2S)$      $\eta_{c}(1S)$      1  -0.0449   7.8    -0.351      10.6   
                $\chi_{c0}(1P)$  28.8    -18.6    22     -14.2      23.6   
                $\chi_{c1}(1P)$  28.7      -35    45     -54.9      47.3   
                $\chi_{c2}(1P)$    28     71.3    46       117        47   
                $\eta_{c}(2S)$  0.206    -21.2  0.19     -19.6     0.233   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -98.9    99     -74.9       131   
$\chi_{c1}(2P)$ $\psi(2S)$        nan     -100   155      -141       110   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      139   150       130       160   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.503   9.1    -0.425      10.8   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -3.77    74     -3.36      83.2   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     9.08    76      8.09      85.4   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.479  
                $\chi_{c0}(1P)$       -15.2  
                $\chi_{c1}(1P)$       -57.7  
                $\chi_{c2}(1P)$         120  
                $\eta_{c}(2S)$          -24  
$\chi_{c0}(2P)$ $\psi(2S)$            -98.9  
$\chi_{c1}(2P)$ $\psi(2S)$             -100  
$\chi_{c2}(2P)$ $\psi(2S)$              139  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.503  
$\chi_{c1}(3P)$ $\psi(2S)$            -3.77  
$\chi_{c2}(3P)$ $\psi(2S)$             9.08

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$      1  -0.0449   7.8    -0.351         10.6   
                $\chi_{c0}(1P)$  28.8    -18.6    22     -14.2         28.7   
                $\chi_{c1}(1P)$  28.7    -32.9    45     -51.6         50.3   
                $\chi_{c2}(1P)$    28     77.6    46       128         44.9   
                $\eta_{c}(2S)$  0.206    -21.2  0.19     -19.6        0.233   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -87.9    99     -74.9          116   
$\chi_{c1}(2P)$ $\psi(2S)$        nan     -100   155      -150          104   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      131   150       113          174   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.378   9.1    -0.425         8.09   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -3.77    74     -3.97         70.3   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     8.15    76      6.14          101   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.479  
                $\chi_{c0}(1P)$       -18.5  
                $\chi_{c1}(1P)$       -57.7  
                $\chi_{c2}(1P)$         124  
                $\eta_{c}(2S)$          -24  
$\chi_{c0}(2P)$ $\psi(2S)$            -87.9  
$\chi_{c1}(2P)$ $\psi(2S)$             -100  
$\chi_{c2}(2P)$ $\psi(2S)$              131  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.378  
$\chi_{c1}(3P)$ $\psi(2S)$            -3.77  
$\chi_{c2}(3P)$ $\psi(2S)$             8.15

'Approximation: ELW'

,$SR$
$SR-PDG$,-22.9
$SR-Deng$,-48.9
$SR-\Gamma$,-1.84


16.6

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\chi_{c0}(1P)$ 28.8    -18.6    22     -14.2          33.5   
                $\chi_{c1}(1P)$ 28.7    -32.9    45     -51.6          54.7   
                $\chi_{c2}(1P)$   28     77.6    46       128          47.5   
$\chi_{c0}(2P)$ $\psi(2S)$       nan    -96.7    99     -74.9           128   
$\chi_{c1}(2P)$ $\psi(2S)$       nan     -112   155      -150           116   
$\chi_{c2}(2P)$ $\psi(2S)$       nan      156   150       113           207   
$\chi_{c0}(3P)$ $\psi(2S)$       nan -0.00185   9.1    -0.425        0.0396   
$\chi_{c1}(3P)$ $\psi(2S)$       nan    -2.15    74     -3.97          40.1   
$\chi_{c2}(3P)$ $\psi(2S)$       nan     6.07    76      6.14          75.1   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\chi_{c0}(1P)$       -21.6  
                $\chi_{c1}(1P)$       -62.8  
                $\chi_{c2}(1P)$         132  
$\chi_{c0}(2P)$ $\psi(2S)$            -96.7  
$\chi_{c1}(2P)$ $\psi(2S)$             -112  
$\chi_{c2}(2P)$ $\psi(2S)$              156  
$\chi_{c0}(3P)$ $\psi(2S)$         -0.00185  
$\chi_{c1}(3P)$ $\psi(2S)$            -2.15  
$\chi_{c2}(3P)$ $\psi(2S)$             6.07

In [42]:
reportSumrules("psi_2S", "lin", "c-lin", sigma=16.6)

'Approximation: tot'

,$SR$
$SR-PDG$,-51.4
$SR-Deng$,-86.8
$SR-\Gamma$,-27.9


16.6

PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi(2S)$      $\eta_{c}(1S)$      1  -0.0449  8.08    -0.363      9.98   
                $\chi_{c0}(1P)$  28.8    -18.6    22     -14.2      24.4   
                $\chi_{c1}(1P)$  28.7    -34.9    42     -51.1      44.4   
                $\chi_{c2}(1P)$    28     71.6    38      97.3      43.9   
                $\eta_{c}(2S)$  0.206    -21.2  0.19     -19.6     0.211   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -85.8   121     -91.5       113   
$\chi_{c1}(2P)$ $\psi(2S)$        nan    -89.2   139      -127      97.8   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      123   135       116       142   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.516    17    -0.493      17.8   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -2.98    94     -2.61       107   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     7.66    97      6.51       114   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.449  
                $\chi_{c0}(1P)$       -15.7  
                $\chi_{c1}(1P)$       -53.9  
                $\chi_{c2}(1P)$         112  
                $\eta_{c}(2S)$        -21.7  
$\chi_{c0}(2P)$ $\psi(2S)$            -85.8  
$\chi_{c1}(2P)$ $\psi(2S)$            -89.2  
$\chi_{c2}(2P)$ $\psi(2S)$              123  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.516  
$\chi_{c1}(3P)$ $\psi(2S)$            -2.98  
$\chi_{c2}(3P)$ $\psi(2S)$             7.66

'Approximation: E1'

...

PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$      1  -0.0449  8.08    -0.363         9.98   
                $\chi_{c0}(1P)$  28.8    -18.6    22     -14.2         29.3   
                $\chi_{c1}(1P)$  28.7    -32.9    42     -48.2           47   
                $\chi_{c2}(1P)$    28     77.6    38       105           42   
                $\eta_{c}(2S)$  0.206    -21.2  0.19     -19.6        0.211   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -76.7   121     -91.5          101   
$\chi_{c1}(2P)$ $\psi(2S)$        nan    -89.2   139      -135           92   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      116   135       102          154   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.376    17    -0.493           13   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -2.98    94     -3.15           89   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     6.81    97      4.81          137   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.449  
                $\chi_{c0}(1P)$       -18.9  
                $\chi_{c1}(1P)$       -53.9  
                $\chi_{c2}(1P)$         116  
                $\eta_{c}(2S)$        -21.7  
$\chi_{c0}(2P)$ $\psi(2S)$            -76.7  
$\chi_{c1}(2P)$ $\psi(2S)$            -89.2  
$\chi_{c2}(2P)$ $\psi(2S)$              116  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.376  
$\chi_{c1}(3P)$ $\psi(2S)$            -2.98  
$\chi_{c2}(3P)$ $\psi(2S)$             6.81

'Approximation: ELW'

,$SR$
$SR-PDG$,-17.5
$SR-Deng$,-80.6
$SR-\Gamma$,-1.12


16.6

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\chi_{c0}(1P)$ 28.8    -18.6    22     -14.2          34.1   
                $\chi_{c1}(1P)$ 28.7    -32.9    42     -48.2            51   
                $\chi_{c2}(1P)$   28     77.6    38       105          44.4   
$\chi_{c0}(2P)$ $\psi(2S)$       nan    -83.9   121     -91.5           111   
$\chi_{c1}(2P)$ $\psi(2S)$       nan      -99   139      -135           102   
$\chi_{c2}(2P)$ $\psi(2S)$       nan      136   135       102           181   
$\chi_{c0}(3P)$ $\psi(2S)$       nan  -0.0135    17    -0.493         0.466   
$\chi_{c1}(3P)$ $\psi(2S)$       nan    -1.26    94     -3.15          37.5   
$\chi_{c2}(3P)$ $\psi(2S)$       nan      4.3    97      4.81          86.7   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\chi_{c0}(1P)$         -22  
                $\chi_{c1}(1P)$       -58.5  
                $\chi_{c2}(1P)$         123  
$\chi_{c0}(2P)$ $\psi(2S)$            -83.9  
$\chi_{c1}(2P)$ $\psi(2S)$              -99  
$\chi_{c2}(2P)$ $\psi(2S)$              136  
$\chi_{c0}(3P)$ $\psi(2S)$          -0.0135  
$\chi_{c1}(3P)$ $\psi(2S)$            -1.26  
$\chi_{c2}(3P)$ $\psi(2S)$              4.3

### $\psi(3S)$

In [43]:
reportSumrules("psi_3S", "scr", "c-scr")

'Approximation: tot'

,$SR$
$SR-PDG$,96.3
$SR-Deng$,149
$SR-\Gamma$,96.3


21.870617716284215

PDG  $SR-PDG$  Deng  $SR-Deng$  $\Gamma$  \
In         Out                                                         
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.149  6.76     -0.101      9.95   
           $\chi_{c0}(1P)$  nan    -0.473   6.7     -0.404      7.85   
           $\chi_{c1}(1P)$  nan    -0.426   6.7     -0.382      7.47   
           $\chi_{c2}(1P)$  nan     0.117   2.5      0.111      2.62   
           $\eta_{c}(2S)$   nan    -0.458  1.78      -0.37       2.2   
           $\chi_{c0}(2P)$  nan     -50.9    27        -60      22.9   
           $\chi_{c1}(2P)$  nan      -157    67      -93.1       113   
           $\chi_{c2}(2P)$  nan       330    82        328      82.4   
           $\eta_{c}(3S)$   nan     -24.2 0.088      -24.5     0.087   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.149  
           $\chi_{c0}(1P)$       -0.473  
           $\chi_{c1}(1P)$       -0.426  
           $\chi_{c2}(1P)$        0.117  
           $\eta_{c}(2S)$        -0.458  
           $\chi_{c0}(2P)$        -50.9  
           $\chi_{c1}(2P)$         -157  
           $\chi_{c2}(2P)$          330  
           $\eta_{c}(3S)$         -24.2

'Approximation: E1'

...

PDG  $SR-PDG$  Deng  $SR-Deng$  E1-$\Gamma$  \
In         Out                                                            
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.149  6.76     -0.101         9.95   
           $\chi_{c0}(1P)$  nan    -0.747   6.7     -0.404         12.4   
           $\chi_{c1}(1P)$  nan    -0.426   6.7     -0.321         8.89   
           $\chi_{c2}(1P)$  nan     0.137   2.5      0.154         2.23   
           $\eta_{c}(2S)$   nan    -0.458  1.78      -0.37          2.2   
           $\chi_{c0}(2P)$  nan     -57.8    27        -60           26   
           $\chi_{c1}(2P)$  nan      -157    67      -87.8          120   
           $\chi_{c2}(2P)$  nan       341    82        353         79.2   
           $\eta_{c}(3S)$   nan     -24.2 0.088      -24.5        0.087   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.149  
           $\chi_{c0}(1P)$       -0.747  
           $\chi_{c1}(1P)$       -0.426  
           $\chi_{c2}(1P)$        0.137  
           $\eta_{c}(2S)$        -0.458  
           $\chi_{c0}(2P)$        -57.8  
           $\chi_{c1}(2P)$         -157  
           $\chi_{c2}(2P)$          341  
           $\eta_{c}(3S)$         -24.2

'Approximation: ELW'

,$SR$
$SR-PDG$,119
$SR-Deng$,204
$SR-\Gamma$,119


37.165665490716144

PDG  $SR-PDG$  Deng  $SR-Deng$  ELW-$\Gamma$  \
In         Out                                                             
$\psi(3S)$ $\chi_{c0}(1P)$  nan    -0.491   6.7     -0.404          8.15   
           $\chi_{c1}(1P)$  nan   -0.0361   6.7     -0.321         0.753   
           $\chi_{c2}(1P)$  nan     0.123   2.5      0.154             2   
           $\chi_{c0}(2P)$  nan     -66.3    27        -60          29.8   
           $\chi_{c1}(2P)$  nan      -177    67      -87.8           135   
           $\chi_{c2}(2P)$  nan       363    82        353          84.4   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\chi_{c0}(1P)$       -0.491  
           $\chi_{c1}(1P)$      -0.0361  
           $\chi_{c2}(1P)$        0.123  
           $\chi_{c0}(2P)$        -66.3  
           $\chi_{c1}(2P)$         -177  
           $\chi_{c2}(2P)$          363

In [44]:
reportSumrules("psi_3S", "lin", "c-lin")

'Approximation: tot'

,$SR$
$SR-PDG$,92.9
$SR-Deng$,150
$SR-\Gamma$,92.9


18.076940202622353

PDG  $SR-PDG$  Deng  $SR-Deng$  $\Gamma$  \
In         Out                                                         
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.135  6.66    -0.0995      9.06   
           $\chi_{c0}(1P)$  nan    -0.413   5.9     -0.356      6.86   
           $\chi_{c1}(1P)$  nan     -0.25     4     -0.226      4.43   
           $\chi_{c2}(1P)$  nan   0.00797  0.25     0.0113     0.176   
           $\eta_{c}(2S)$   nan    -0.385  1.65     -0.343      1.85   
           $\chi_{c0}(2P)$  nan     -45.9    19      -42.2      20.6   
           $\chi_{c1}(2P)$  nan      -128    55      -76.2      92.3   
           $\chi_{c2}(2P)$  nan       268    67        269      66.7   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.135  
           $\chi_{c0}(1P)$       -0.413  
           $\chi_{c1}(1P)$        -0.25  
           $\chi_{c2}(1P)$      0.00797  
           $\eta_{c}(2S)$        -0.385  
           $\chi_{c0}(2P)$        -45.9  
           $\chi_{c1}(2P)$         -128  
           $\chi_{c2}(2P)$          268

'Approximation: E1'

...

PDG  $SR-PDG$  Deng  $SR-Deng$  E1-$\Gamma$  \
In         Out                                                            
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.135  6.66    -0.0995         9.06   
           $\chi_{c0}(1P)$  nan    -0.637   5.9     -0.356         10.6   
           $\chi_{c1}(1P)$  nan     -0.25     4     -0.192         5.23   
           $\chi_{c2}(1P)$  nan   0.00928  0.25     0.0154        0.151   
           $\eta_{c}(2S)$   nan    -0.385  1.65     -0.343         1.85   
           $\chi_{c0}(2P)$  nan     -51.7    19      -42.2         23.3   
           $\chi_{c1}(2P)$  nan      -128    55      -72.1         97.6   
           $\chi_{c2}(2P)$  nan       276    67        288         64.3   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.135  
           $\chi_{c0}(1P)$       -0.637  
           $\chi_{c1}(1P)$        -0.25  
           $\chi_{c2}(1P)$      0.00928  
           $\eta_{c}(2S)$        -0.385  
           $\chi_{c0}(2P)$        -51.7  
           $\chi_{c1}(2P)$         -128  
           $\chi_{c2}(2P)$          276

'Approximation: ELW'

,$SR$
$SR-PDG$,91.8
$SR-Deng$,173
$SR-\Gamma$,91.8


28.71714994909823

PDG  $SR-PDG$  Deng  $SR-Deng$  ELW-$\Gamma$  \
In         Out                                                             
$\psi(3S)$ $\chi_{c0}(1P)$  nan    -0.375   5.9     -0.356          6.21   
           $\chi_{c1}(1P)$  nan  -0.00421     4     -0.192        0.0879   
           $\chi_{c2}(1P)$  nan     0.343  0.25     0.0154          5.56   
           $\chi_{c0}(2P)$  nan     -58.4    19      -42.2          26.3   
           $\chi_{c1}(2P)$  nan      -142    55      -72.1           109   
           $\chi_{c2}(2P)$  nan       293    67        288            68   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\chi_{c0}(1P)$       -0.375  
           $\chi_{c1}(1P)$     -0.00421  
           $\chi_{c2}(1P)$        0.343  
           $\chi_{c0}(2P)$        -58.4  
           $\chi_{c1}(2P)$         -142  
           $\chi_{c2}(2P)$          293

In [45]:
reportSumrules("psi_1_1D", "scr", "c-scr", sigma=[0]*3)

'Approximation: tot'

,$SR$
$SR-PDG$,-243
$SR-Deng$,-263
$SR-\Gamma$,-298


21.904877618240572

PDG  $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   -0.0124   nan   -0.0124     0.386   
                $\chi_{c0}(1P)$  188     -52.3   261     -72.7       291   
                $\chi_{c1}(1P)$ 67.7     -21.2   135     -42.3       153   
                $\chi_{c2}(1P)$  nan      5.69   8.1      5.73      8.04   
                $\eta_{c}(2S)$   nan -0.000483   nan -0.000483   0.00011   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan      -139    12     -80.7      20.7   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     -36.4   9.8     -73.4      4.86   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      1.16  0.46      1.02     0.522   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.0282  0.39   -0.0343      0.32   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     -0.25     2     -0.22      2.28   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   -0.0455  0.79   -0.0386      0.93   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0124  
                $\chi_{c0}(1P)$       -81.1  
                $\chi_{c1}(1P)$       -48.1  
                $\chi_{c2}(1P)$        5.69  
                $\eta_{c}(2S)$    -0.000483  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -139  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -36.4  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.16  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$      -0.0282  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$        -0.25  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      -0.0455

'Approximation: E1'

...

PDG  $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   -0.0124   nan   -0.0124        0.386   
                $\chi_{c0}(1P)$  188     -52.3   261     -72.7          259   
                $\chi_{c1}(1P)$ 67.7     -22.7   135     -45.3          129   
                $\chi_{c2}(1P)$  nan      3.42   8.1      4.67         5.93   
                $\eta_{c}(2S)$   nan -0.000483   nan -0.000483      0.00011   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan      -144    12     -80.7         21.3   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     -37.9   9.8     -71.5         5.19   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      1.94  0.46      1.41        0.632   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan    -0.032  0.39   -0.0343        0.364   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan    -0.289     2    -0.189         3.05   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan    0.0314  0.79      0.11        0.225   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0124  
                $\chi_{c0}(1P)$         -72  
                $\chi_{c1}(1P)$       -43.2  
                $\chi_{c2}(1P)$        3.42  
                $\eta_{c}(2S)$    -0.000483  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -144  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -37.9  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.94  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.032  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.289  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$       0.0314

'Approximation: ELW'

,$SR$
$SR-PDG$,-258
$SR-Deng$,-264
$SR-\Gamma$,-321


26.309758527106396

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\chi_{c0}(1P)$  188    -52.3   261     -72.7           318   
                $\chi_{c1}(1P)$ 67.7    -22.7   135     -45.3           148   
                $\chi_{c2}(1P)$  nan     3.81   8.1      4.67          6.61   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     -149    12     -80.7          22.1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -39.6   9.8     -71.5          5.42   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     2.12  0.46      1.41         0.692   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.835  0.39   -0.0343          9.49   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan  -0.0371     2    -0.189         0.392   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan    0.011  0.79      0.11        0.0791   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\chi_{c0}(1P)$       -88.5  
                $\chi_{c1}(1P)$       -49.6  
                $\chi_{c2}(1P)$        3.81  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -149  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -39.6  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.12  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.835  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$      -0.0371  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$        0.011

In [46]:
reportSumrules("psi_1_1D", "lin", "c-lin")

'Approximation: tot'

,$SR$
$SR-PDG$,-209
$SR-Deng$,-307
$SR-\Gamma$,-263


11.353673379227843

PDG  $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan    -0.011   nan    -0.011     0.341   
                $\chi_{c0}(1P)$  188     -52.3   272     -75.8       296   
                $\chi_{c1}(1P)$ 67.7     -21.3   138     -43.4       145   
                $\chi_{c2}(1P)$  nan      5.28   7.1      4.99       7.5   
                $\eta_{c}(2S)$   nan -0.000348   nan -0.000348  7.92e-05   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan      -112    20      -134      16.6   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     -28.9   7.9     -59.1      3.87   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     0.924  0.36     0.827     0.402   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  -0.00781  0.27   -0.0132      0.16   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan    -0.223   3.2    -0.204      3.49   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   -0.0533   1.5   -0.0454      1.76   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$       -0.011  
                $\chi_{c0}(1P)$       -82.3  
                $\chi_{c1}(1P)$       -45.6  
                $\chi_{c2}(1P)$        5.28  
                $\eta_{c}(2S)$    -0.000348  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -112  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -28.9  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$        0.924  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$     -0.00781  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.223  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      -0.0533

'Approximation: E1'

...

PDG  $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan    -0.011   nan    -0.011        0.341   
                $\chi_{c0}(1P)$  188     -52.3   272     -75.8          264   
                $\chi_{c1}(1P)$ 67.7     -22.7   138     -46.3          123   
                $\chi_{c2}(1P)$  nan      3.24   7.1      4.09         5.62   
                $\eta_{c}(2S)$   nan -0.000348   nan -0.000348     7.92e-05   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan      -115    20      -134         17.1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan       -30   7.9     -57.6         4.12   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      1.49  0.36      1.11        0.484   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  -0.00907  0.27   -0.0132        0.186   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan    -0.262   3.2    -0.171         4.91   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan    0.0265   1.5     0.116        0.343   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$       -0.011  
                $\chi_{c0}(1P)$       -73.5  
                $\chi_{c1}(1P)$       -41.2  
                $\chi_{c2}(1P)$        3.24  
                $\eta_{c}(2S)$    -0.000348  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -115  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          -30  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.49  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$     -0.00907  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.262  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$       0.0265

'Approximation: ELW'

,$SR$
$SR-PDG$,-221
$SR-Deng$,-309
$SR-\Gamma$,-282


11.41197681048496

PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\chi_{c0}(1P)$  188    -52.3   272     -75.8           321   
                $\chi_{c1}(1P)$ 67.7    -22.7   138     -46.3           140   
                $\chi_{c2}(1P)$  nan      3.6   7.1      4.09          6.24   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     -119    20      -134          17.7   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -31.3   7.9     -57.6          4.29   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     1.62  0.36      1.11         0.527   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.452  0.27   -0.0132          9.27   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan  -0.0184   3.2    -0.171         0.345   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.00708   1.5     0.116        0.0916   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\chi_{c0}(1P)$       -89.5  
                $\chi_{c1}(1P)$         -47  
                $\chi_{c2}(1P)$         3.6  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$         -119  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -31.3  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.62  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.452  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$      -0.0184  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      0.00708

## Sumrules for the article

In [11]:
def getWidths(pname, prefix, cfgname):
    my_E1 = getRs(pname, prefix, cfgname, suffix="E1")
    my_E1.rename(columns={"$\Gamma$": "E1", "$r^{(0)}$": "r0", "$r^{(2)}$": "r2"}, inplace=True)
    deng = pd.concat([getDengTrans(prefix, cfg) for\
                          cfg in data_pack[cfgname.split("-")[0]]
                     ], sort=False, axis=0)
    pdg, pdg_err = getPDGTrans(cfgname.split("-")[0])
    pdg = pdg.reindex(my_E1.index)
    pdg_err = pdg_err.reindex(my_E1.index)
    deng = deng.drop(columns="Deng").reindex(my_E1.index.values)
    deng["E1_min"] = deng.join(my_E1[["E1"]]).min(axis=1)
    deng["E1_max"] = deng.join(my_E1[["E1"]]).max(axis=1)
    deng = deng[["E1_min", "E1_max"]]
    res = pd.concat([pdg, my_E1, deng], axis=1)
    err = res*sp.nan
    err["PDG"] = pdg_err
    return res, err.abs()

In [12]:
def getSumrule(pname, prefix, cfgname):
    pretty_pname = "${}$".format(code2name(pname))
    
    widths, widths_err = getWidths(pname, prefix, cfgname)
    masses = getMasses(prefix, cfgname)
    pprops = getPprops(prefix, cfgname)
    
    sumrule_prefactors = []
    
    abovethr = widths.query("Out == r'{}'".format(pretty_pname))
    abovethr_pcls = [p[0] for p in abovethr.index.values]
    sumrule_prefactors.append(
        8*sp.pi**2*masses.loc[abovethr_pcls, "E"]**3\
        /(masses.loc[abovethr_pcls, "E"]**2 - masses.loc[pretty_pname, "E"]**2)**3\
        *pprops.loc[abovethr_pcls, "xJ"]
    )
    sumrule_prefactors[0].index = pd.Index([(v, pretty_pname) for v in sumrule_prefactors[0].index.values])
    
    belowthr = widths.query("In == r'{}'".format(pretty_pname))
    belowthr_pcls = [p[1] for p in belowthr.index.values]
    sumrule_prefactors.append(
        8*sp.pi**2*masses.loc[pretty_pname, "E"]**3\
        /(masses.loc[pretty_pname, "E"]**2 - masses.loc[belowthr_pcls, "E"]**2)**3\
        *pprops.loc[pretty_pname, "xJ"]
    )
    sumrule_prefactors[1].index = pd.Index([(pretty_pname, v) for v in sumrule_prefactors[1].index.values])
    
    sumrule_prefactors_cnct = pd.concat(sumrule_prefactors, axis=0)
    sumrule_prefactors_cnct = sumrule_prefactors_cnct.reindex(widths.index.values)
    
    sumrule_prefactors_cnct = sumrule_prefactors_cnct*(widths.loc[:, "r2"] - widths.loc[:, "r0"])\
                *(0.19732697)**2*10**4*10**3
    sumrule = widths.drop(columns=["r0", "r2"])\
                .apply(\
                        lambda s: s*sumrule_prefactors_cnct.values\
                    )
    sumrule_err = widths_err.drop(columns=["r0", "r2"])\
                .apply(\
                        lambda s: s*sumrule_prefactors_cnct.values\
                    )
    
    widths.drop(columns=["r2", "r0"], inplace=True)
    widths = dfsort(widths, lambda a,b: cmpMultyStatesByCode(prefix, cfgname
                                                         ,[name2code(i[1:-1]) for i in a]
                                                         ,[name2code(i[1:-1]) for i in b]))
    sumrule = sumrule.reindex(widths.index.values)
    sumrule_err = sumrule_err.reindex(widths.index.values) 
    
    masses = dfsort(\
                      masses.loc[belowthr_pcls+abovethr_pcls, :]
                    , lambda a, b: \
                        cmpStatesByName(prefix, cfgname, a[1:-1], b[1:-1]) \
                   ).rename(columns={"E": "M"})
    
    return masses, widths, widths_err.abs(), sumrule, sumrule_err.abs()

In [43]:
def sumruleReport(pname, prefix, cfgname):
    pretty_pname = "${}$".format(code2name(pname))
    def istriv(val):
        return sp.isnan(val) or val == 0
    
    def numfmt(num):
        return "{: .5f}".format(num)
    
    def quartet(a, amin, amax, b, sign=2):
        if not istriv(b):
            prec = -int(sp.floor(sp.log10(b)))+sign-1 # precision for 2 sign figure of b
        else:
            prec = -int(sp.floor(sp.log10(sp.absolute(amax-amin))))+sign-1 # if b is triv => prec = 2 sign figures of amax-amin
        
        return prec, sp.around(sp.array([a, amin, amax, b]), decimals=prec)
    
    def quartet_str(a, amin, amax, b, sign=2):
        prec, (a, amin, amax, b) = quartet(a, amin, amax, b, sign=sign)
        if prec > 0:
            patt = r"{{:.{:d}f}}".format(prec)
        else:
            patt = r"{:d}"
            a = int(a)
            amin = int(amin)
            amax = int(amax)
        respatt = "${{"+patt+"}}_{{-"+patt+"}}^{{+"+patt+"}}$"
        if not istriv(b):
            respatt += r"$\pm{{{:."+str(sign)+"g}}}_{{exp}}$"
        return respatt.format(a, sp.absolute(amin), amax, b)
    
    def doublet(a, b, sign=2):
        prec, (a, amin, amax, b) = quartet(a, 0, a, b) 
        return prec, sp.array([a, b])
    
    def doublet_str(a, b, sign=2):
        prec, (a, b) = doublet(a, b, sign=sign)
        if prec > 0:
            patt = r"{{:.{:d}f}}".format(prec)
        else:
            patt = r"{:d}"
            a = int(a)
        respatt = r"${{"+patt+"}}$"
        if not istriv(b):
            respatt += r"$\pm{{{:."+str(sign)+"g}}}_{{exp}}$"
        return respatt.format(a, b)
    
    def fillsubtot(row):
        oldpdg = row["PDG"]
        for source in row.index.values:
            if istriv(oldpdg):
                
                if not istriv(row[source]):
                    continue

                if istriv(row["E1"]):
                    row[source] = 0
                else:
                    row[source] = row["E1"]
            else:
                row[source] = row["PDG"]
        return row
    
    def spreads(triple):
        return (triple[0], min(triple)-triple[0], max(triple)-triple[0])
    
    masses, widths, widths_err, sumrule, sumrule_err = getSumrule(pname, prefix, cfgname)
    sumrule[["E1_min", "E1_max"]] = sumrule[["E1_min", "E1_max"]].apply(lambda row: sorted(row), axis=1, result_type="expand").rename(columns={0: "E1_min", 1: "E1_max"})
    
    abovethr = [pair[0] for pair in widths.query("Out == r'{}'".format(pretty_pname)).index.values]
    
    tab = LongTable("|c|c|c|c|c|")

    # Header
    tab.add_hline()
    tab.add_row([  "X"
                 , NoEscape(r"$m_X$ [$MeV$]")\
                 , NoEscape(r"$\rightleftarrows$")\
                 , NoEscape(r"$\Gamma(X \leftrightarrows {})$ [$KeV$]".format(code2name(pname)))\
                 , NoEscape(r"$SR_1$ [$\mu b$]")
                ])
    tab.add_hline()

    curN = "1"
    tot = sumrule.iloc[0].fillna(0)*0
    tot_err = sumrule_err.iloc[0].fillna(0)*0
    getN = re.compile("\((\d+)[A-Z]\)\$$")
    for label in masses.index.values:
        if label in abovethr:
            label_pair = (label, pretty_pname)
        else:
            label_pair = (pretty_pname, label)

        # Manage subtotals
        newN = getN.search(label).groups()[0]
        if newN != curN:
            tab.add_hline()
            
            srentry = quartet_str( *spreads(tot[["PDG", "E1_min", "E1_max"]])
                                     , sp.sqrt(tot_err["PDG"]))
            
            tab.add_row([NoEscape(i) for i in ["Subtotal"] + [""]*3 + [srentry]])
            tab.add_hline()
            tab.add_hline()
            tot = fillsubtot(sumrule.loc[label_pair].copy())
            tot_err = fillsubtot(sumrule_err.loc[label_pair].copy()**2)
            curN = newN
        else:
            tot += fillsubtot(sumrule.loc[label_pair].copy())
            tot_err += fillsubtot(sumrule_err.loc[label_pair].copy()**2)
            
        # Regular entry
        curdata = widths.loc[label_pair, :]
        
        srentry = sumrule.loc[label_pair]["PDG"]
        if istriv(srentry):
            srentry = quartet_str( *spreads(sumrule.loc[label_pair, ["E1", "E1_min", "E1_max"]])
                                 , sumrule_err.loc[label_pair, "PDG"])
        else:
            srentry = doublet_str(srentry, sumrule_err.loc[label_pair, "PDG"])
        
        dwentry = widths.loc[label_pair]["PDG"]
        if istriv(dwentry):
            dwentry = quartet_str( *spreads(widths.loc[label_pair, ["E1", "E1_min", "E1_max"]])
                                 , widths_err.loc[label_pair, "PDG"])
        else:
            dwentry = doublet_str(dwentry, widths_err.loc[label_pair, "PDG"])

        
        arrow = r"$\rightarrow$" if label in abovethr else r"$\leftarrow$"

        tab.add_row([NoEscape(i) for i in [label, numfmt(masses.loc[label, "M"]), arrow, dwentry, srentry]])
        tab.add_hline()

    # Final subtotal
    tab.add_hline()
    
    srentry = quartet_str( *spreads(tot[["PDG", "E1_min", "E1_max"]])
                                     , sp.sqrt(tot_err["PDG"]))
    
    tab.add_row([NoEscape(i) for i in ["Subtotal"] + [""]*3 + [srentry]])
    tab.add_hline()
    tab.add_hline()

    # Total
    tot = sumrule.apply(fillsubtot, axis=1).sum(axis=0)
    tot_err = sumrule_err.apply(sp.square).sum(axis=0)
    
    srentry = quartet_str( *spreads(tot[["PDG", "E1_min", "E1_max"]])
                                     , sp.sqrt(tot_err["PDG"]))
    
    tab.add_row([NoEscape(i) for i in ["Total"] + [""]*3 + [srentry]])
    tab.add_hline()
    
    return tab

In [34]:
def doReport(pname, prefix, cfgname):
    masses, widths, widths_err, sr, sr_err = getSumrule(pname, prefix, cfgname)
    display(sr.sum(axis=0))
    display(widths)
    display(sr)
    sumruleReport(pname, prefix, cfgname).generate_tex("export/sr-article.{}.{}".format(pname, cfgname))

### Bottomonium sumrules

In [39]:
doReport("yps_1S", "scr", "b-scr")

PDG            0
E1         0.399
E1_min   -0.0517
E1_max     -0.11
dtype: float64

,,PDG,E1,E1_min,E1_max
In,Out,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,24.2,23.8,29.9
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,30.2,29.5,36.6
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,36.1,32.6,40.2
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,4.44,2.5,11.4
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.7,5.5,12.4
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,16.9,8.02,16.9
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.37,0.3,6.41
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,5.55,1.3,6.8
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,10.7,2.8,10.7


,,PDG,E1,E1_min,E1_max
In,Out,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.55,-1.53,-1.92
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.3,-2.25,-2.79
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,4.03,3.64,4.48
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0416,-0.0234,-0.107
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.138,-0.071,-0.16
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.348,0.165,0.348
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00561,-0.00122,-0.0261
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0321,-0.00751,-0.0393
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,0.0988,0.0258,0.0988


In [37]:
doReport("yps_2S", "scr", "b-scr")

PDG      5.71
E1       65.9
E1_min   52.1
E1_max   72.5
dtype: float64

PDG      E1  E1_min E1_max
In              Out                                         
$\Upsilon(2S)$  $\chi_{b0}(1P)$ -3.27   -3.19   -2.45  -4.49
                $\chi_{b1}(1P)$ -5.84   -6.02   -4.31  -6.72
                $\chi_{b2}(1P)$  9.79      11    8.05   11.2
$\chi_{b0}(2P)$ $\Upsilon(2S)$    NaN   -5.72   -4.72  -5.72
$\chi_{b1}(2P)$ $\Upsilon(2S)$    NaN   -7.27   -6.34  -7.58
$\chi_{b2}(2P)$ $\Upsilon(2S)$    NaN    11.2    9.57   12.7
$\chi_{b0}(3P)$ $\Upsilon(2S)$    NaN -0.0872 -0.0681 -0.192
$\chi_{b1}(3P)$ $\Upsilon(2S)$    NaN  -0.267  -0.164  -0.29
$\chi_{b2}(3P)$ $\Upsilon(2S)$    NaN   0.602   0.361  0.602

PDG   E1  E1_min  E1_max
In              Out                                      
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22 1.19    0.91    1.67
                $\chi_{b1}(1P)$ 2.21 2.28    1.63    2.54
                $\chi_{b2}(1P)$ 2.29 2.58    1.88    2.62
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan 13.2    10.9    13.2
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan 15.3    13.3    15.9
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan 16.7    14.2    18.8
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan 2.18     1.7     4.8
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan 5.03     3.1    5.48
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan  7.5     4.5     7.5

KeyError: ('PDG', 'dE1_min', 'dE1_max')

In [53]:
doReport("yps_3S", "scr", "b-scr")

PDG     0.716
FULL   -0.553
LWA    -0.123
Deng    -1.88
E1av    -1.92
dtype: float64

PDG     FULL    LWA   Deng    E1av
In              Out                                                  
$\Upsilon(3S)$  $\eta_{b}(1S)$  0.0104   0.0795    nan  0.071   0.041
                $\chi_{b0}(1P)$ 0.0549   0.0903 0.0436  0.097    0.08
                $\chi_{b1}(1P)$ 0.0183 8.08e-05 0.0413 0.0005  0.0853
                $\chi_{b2}(1P)$  0.201     0.16  0.455   0.14   0.671
                $\eta_{b}(2S)$     nan   0.0122    nan  0.011 0.00559
                $\chi_{b0}(2P)$    1.2     1.23    1.3   1.21    1.43
                $\chi_{b1}(2P)$   2.56     2.58   2.65   2.61    2.44
                $\chi_{b2}(2P)$   2.66     3.21   3.23   3.16    2.76
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan     7.93   7.98   7.95     7.7
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan     9.55   9.84   10.3    10.2
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan     10.6   11.5   10.8    12.4

PDG      FULL      LWA      Deng  \
In              Out                                                      
$\Upsilon(3S)$  $\eta_{b}(1S)$  -0.000157  -0.00121      NaN  -0.00108   
                $\chi_{b0}(1P)$  -0.00558  -0.00919 -0.00443  -0.00987   
                $\chi_{b1}(1P)$   -0.0012 -5.28e-06  -0.0027 -3.27e-05   
                $\chi_{b2}(1P)$    0.0131    0.0104   0.0296    0.0091   
                $\eta_{b}(2S)$        NaN  -0.00327      NaN  -0.00296   
                $\chi_{b0}(2P)$     -7.62     -7.84    -8.25     -7.69   
                $\chi_{b1}(2P)$     -15.3     -15.4    -15.8     -15.5   
                $\chi_{b2}(2P)$      23.6      28.4     28.6        28   
$\chi_{b0}(3P)$ $\Upsilon(3S)$        NaN     -12.3    -12.4     -12.3   
$\chi_{b1}(3P)$ $\Upsilon(3S)$        NaN     -14.4    -14.8     -15.5   
$\chi_{b2}(3P)$ $\Upsilon(3S)$        NaN      20.9     22.5      21.2   

                                     E1av  
In              Out                        
$\Upsilon(3S)$  $\eta_{b}(1S)$  -0.000623  
                $\chi_{b0}(1P)$  -0.00814  
                $\chi_{b1}(1P)$  -0.00558  
                $\chi_{b2}(1P)$    0.0436  
                $\eta_{b}(2S)$    -0.0015  
                $\chi_{b0}(2P)$     -9.09  
                $\chi_{b1}(2P)$     -14.5  
                $\chi_{b2}(2P)$      24.5  
$\chi_{b0}(3P)$ $\Upsilon(3S)$      -11.9  
$\chi_{b1}(3P)$ $\Upsilon(3S)$      -15.3  
$\chi_{b2}(3P)$ $\Upsilon(3S)$       24.4

###  Charmonium Screened Sumrules

In [72]:
doReport("psi_1S", "scr", "c-scr")

PDG           814
E1       1.26e+03
E1_min        705
E1_max   1.42e+03
dtype: float64

,,PDG,E1,E1_min,E1_max
In,Out,,,,
$\chi_{c0}(1P)$,$\psi(1S)$,-20.9,-19.6,-15.7,-23.8
$\chi_{c1}(1P)$,$\psi(1S)$,-28.1,-30.1,-23.3,-34.6
$\chi_{c2}(1P)$,$\psi(1S)$,45.4,61.4,34.4,61.4
$\chi_{c0}(2P)$,$\psi(1S)$,NaN,-0.0222,-0.0153,-0.868
$\chi_{c1}(2P)$,$\psi(1S)$,NaN,-1.25,-0.238,-1.41
$\chi_{c2}(2P)$,$\psi(1S)$,NaN,5.46,1.24,5.46


,,PDG,E1,E1_min,E1_max
In,Out,,,,
$\chi_{c0}(1P)$,$\psi(1S)$,151,142,114,172
$\chi_{c1}(1P)$,$\psi(1S)$,288,309,239,354
$\chi_{c2}(1P)$,$\psi(1S)$,374,506,284,506
$\chi_{c0}(2P)$,$\psi(1S)$,nan,1.89,1.3,74
$\chi_{c1}(2P)$,$\psi(1S)$,nan,73.3,14,83
$\chi_{c2}(2P)$,$\psi(1S)$,nan,233,53,233


PDG      -20.9
E1       -19.6
E1_min   -15.7
E1_max   -23.8
Name: ($\chi_{c0}(1P)$, $\psi(1S)$), dtype: object

else
else
else
else


...

else
else
else


PDG         NaN
E1        -1.25
E1_min   -0.238
E1_max    -1.41
Name: ($\chi_{c1}(2P)$, $\psi(1S)$), dtype: object

else
else
else


PDG       NaN
E1       5.46
E1_min   1.24
E1_max   5.46
Name: ($\chi_{c2}(2P)$, $\psi(1S)$), dtype: object

else
else
else


In [55]:
doReport("psi_2S", "scr", "c-scr")

PDG    -3.47
FULL   -33.6
LWA     16.3
Deng   -53.5
E1av   -12.8
dtype: float64

PDG  FULL    LWA  Deng  E1av
In              Out                                           
$\psi(2S)$      $\eta_{c}(1S)$      1  10.6    nan   7.8  5.27
                $\chi_{c0}(1P)$  28.8  23.6   33.5    22    48
                $\chi_{c1}(1P)$  28.7  47.3   54.7    45  42.5
                $\chi_{c2}(1P)$    28    47   47.5    46  32.1
                $\eta_{c}(2S)$  0.206 0.233    nan  0.19 0.127
$\chi_{c0}(2P)$ $\psi(2S)$        nan   131    128    99  89.5
$\chi_{c1}(2P)$ $\psi(2S)$        nan   110    116   155   122
$\chi_{c2}(2P)$ $\psi(2S)$        nan   160    207   150   202
$\chi_{c0}(3P)$ $\psi(2S)$        nan  10.8 0.0396   9.1    16
$\chi_{c1}(3P)$ $\psi(2S)$        nan  83.2   40.1    74  51.5
$\chi_{c2}(3P)$ $\psi(2S)$        nan  85.4   75.1    76    73

PDG   FULL      LWA   Deng   E1av
In              Out                                                  
$\psi(2S)$      $\eta_{c}(1S)$  -0.0449 -0.479      NaN -0.351 -0.237
                $\chi_{c0}(1P)$   -18.6  -15.2    -21.6  -14.2    -31
                $\chi_{c1}(1P)$     -35  -57.7    -66.7  -54.9  -51.8
                $\chi_{c2}(1P)$    71.3    120      121    117   81.8
                $\eta_{c}(2S)$    -21.2    -24      NaN  -19.6    -13
$\chi_{c0}(2P)$ $\psi(2S)$          NaN  -98.9    -96.7  -74.9  -67.7
$\chi_{c1}(2P)$ $\psi(2S)$          NaN   -100     -105   -141   -111
$\chi_{c2}(2P)$ $\psi(2S)$          NaN    139      180    130    175
$\chi_{c0}(3P)$ $\psi(2S)$          NaN -0.503 -0.00185 -0.425 -0.748
$\chi_{c1}(3P)$ $\psi(2S)$          NaN  -3.77    -1.82  -3.36  -2.33
$\chi_{c2}(3P)$ $\psi(2S)$          NaN   9.08     7.99   8.09   7.77

In [56]:
doReport("psi_1_1D", "scr", "c-scr")

PDG    -73.5
FULL    -298
LWA     -319
Deng    -263
E1av    -449
dtype: float64

PDG    FULL    LWA  Deng  E1av
In              Out                                            
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   0.386    nan   nan   nan
                $\chi_{c0}(1P)$  188     291    318   261   342
                $\chi_{c1}(1P)$ 67.7     153    148   135   114
                $\chi_{c2}(1P)$  nan    8.04   6.61   8.1   5.7
                $\eta_{c}(2S)$   nan 0.00011    nan   nan   nan
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    20.7   22.1    12  31.5
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    4.86   5.42   9.8  14.9
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan   0.522  0.692  0.46  1.13
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan    0.32   9.49  0.39  4.87
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan    2.28  0.392     2  2.21
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan    0.93 0.0791  0.79  0.95

PDG      FULL      LWA    Deng    E1av
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$    NaN   -0.0124      NaN     NaN     nan
                $\chi_{c0}(1P)$ -52.3     -81.1    -88.5   -72.7   -95.1
                $\chi_{c1}(1P)$ -21.2     -48.1    -46.4   -42.3   -35.6
                $\chi_{c2}(1P)$   NaN      5.69     4.68    5.73    4.03
                $\eta_{c}(2S)$    NaN -0.000483      NaN     NaN     nan
$\chi_{c0}(2P)$ $\psi_{1}(1D)$    NaN      -139     -149   -80.7    -212
$\chi_{c1}(2P)$ $\psi_{1}(1D)$    NaN     -36.4    -40.6   -73.4    -112
$\chi_{c2}(2P)$ $\psi_{1}(1D)$    NaN      1.16     1.54    1.02    2.51
$\chi_{c0}(3P)$ $\psi_{1}(1D)$    NaN   -0.0282   -0.835 -0.0343  -0.428
$\chi_{c1}(3P)$ $\psi_{1}(1D)$    NaN     -0.25  -0.0431   -0.22  -0.243
$\chi_{c2}(3P)$ $\psi_{1}(1D)$    NaN   -0.0455 -0.00387 -0.0386 -0.0465

### Charmonium Linear

In [57]:
doReport("psi_1S", "lin", "c-lin")

PDG     -1.6
FULL   -24.4
LWA       20
Deng   -24.8
E1av   -2.71
dtype: float64

,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.53,nan,2.39,1.98
$\chi_{c0}(1P)$,$\psi(1S)$,151,187,177,172,146
$\chi_{c1}(1P)$,$\psi(1S)$,288,344,367,306,296
$\chi_{c2}(1P)$,$\psi(1S)$,374,325,471,284,378
$\eta_{c}(2S)$,$\psi(1S)$,nan,3.04,nan,2.64,4.71
$\chi_{c0}(2P)$,$\psi(1S)$,nan,5.24,1.66,6.1,37.6
$\chi_{c1}(2P)$,$\psi(1S)$,nan,86.9,46.4,81,51
$\chi_{c2}(2P)$,$\psi(1S)$,nan,116,129,93,86


,,PDG,FULL,LWA,Deng,E1av
In,Out,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,-13.3,-21.3,NaN,-20.2,-16.7
$\chi_{c0}(1P)$,$\psi(1S)$,-20.9,-25.9,-24.4,-23.8,-20.2
$\chi_{c1}(1P)$,$\psi(1S)$,-24.7,-29.4,-31.5,-26.2,-25.4
$\chi_{c2}(1P)$,$\psi(1S)$,57.3,49.7,72.1,43.5,57.9
$\eta_{c}(2S)$,$\psi(1S)$,NaN,-0.093,NaN,-0.0809,-0.145
$\chi_{c0}(2P)$,$\psi(1S)$,NaN,-0.0614,-0.0195,-0.0716,-0.442
$\chi_{c1}(2P)$,$\psi(1S)$,NaN,-1.17,-0.625,-1.09,-0.687
$\chi_{c2}(2P)$,$\psi(1S)$,NaN,3.96,4.38,3.16,2.92


In [58]:
doReport("psi_2S", "lin", "c-lin")

PDG    -3.07
FULL   -27.9
LWA     13.1
Deng   -86.8
E1av   -15.5
dtype: float64

PDG  FULL   LWA  Deng  E1av
In              Out                                          
$\psi(2S)$      $\eta_{c}(1S)$      1  9.98   nan  8.08  5.27
                $\chi_{c0}(1P)$  28.8  24.4  34.1    22    48
                $\chi_{c1}(1P)$  28.7  44.4    51    42  42.5
                $\chi_{c2}(1P)$    28  43.9  44.4    38  32.1
                $\eta_{c}(2S)$  0.206 0.211   nan  0.19 0.127
$\chi_{c0}(2P)$ $\psi(2S)$        nan   113   111   121  89.5
$\chi_{c1}(2P)$ $\psi(2S)$        nan  97.8   102   139   122
$\chi_{c2}(2P)$ $\psi(2S)$        nan   142   181   135   202
$\chi_{c0}(3P)$ $\psi(2S)$        nan  17.8 0.466    17    16
$\chi_{c1}(3P)$ $\psi(2S)$        nan   107  37.5    94  51.5
$\chi_{c2}(3P)$ $\psi(2S)$        nan   114  86.7    97    73

PDG   FULL     LWA   Deng   E1av
In              Out                                                 
$\psi(2S)$      $\eta_{c}(1S)$  -0.0449 -0.449     NaN -0.363 -0.237
                $\chi_{c0}(1P)$   -18.6  -15.7     -22  -14.2    -31
                $\chi_{c1}(1P)$   -34.9  -53.9     -62  -51.1  -51.6
                $\chi_{c2}(1P)$    71.6    112     114   97.3   82.2
                $\eta_{c}(2S)$    -21.2  -21.7     NaN  -19.6    -13
$\chi_{c0}(2P)$ $\psi(2S)$          NaN  -85.8   -83.9  -91.5  -67.7
$\chi_{c1}(2P)$ $\psi(2S)$          NaN  -89.2   -93.1   -127   -111
$\chi_{c2}(2P)$ $\psi(2S)$          NaN    123     156    116    174
$\chi_{c0}(3P)$ $\psi(2S)$          NaN -0.516 -0.0135 -0.493 -0.465
$\chi_{c1}(3P)$ $\psi(2S)$          NaN  -2.98   -1.04  -2.61  -1.43
$\chi_{c2}(3P)$ $\psi(2S)$          NaN   7.66    5.81   6.51    4.9

In [59]:
doReport("psi_1_1D", "lin", "c-lin")

PDG    -73.6
FULL    -263
LWA     -280
Deng    -307
E1av    -448
dtype: float64

PDG     FULL    LWA  Deng  E1av
In              Out                                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan    0.341    nan   nan   nan
                $\chi_{c0}(1P)$  188      296    321   272   342
                $\chi_{c1}(1P)$ 67.7      145    140   138   114
                $\chi_{c2}(1P)$  nan      7.5   6.24   7.1   5.7
                $\eta_{c}(2S)$   nan 7.92e-05    nan   nan   nan
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     16.6   17.7    20  31.5
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     3.87   4.29   7.9  14.9
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan    0.402  0.527  0.36  1.13
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan     0.16   9.27  0.27  4.87
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     3.49  0.345   3.2  2.21
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan     1.76 0.0916   1.5  0.95

PDG      FULL      LWA    Deng    E1av
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$    NaN    -0.011      NaN     NaN     nan
                $\chi_{c0}(1P)$ -52.3     -82.3    -89.5   -75.8   -95.1
                $\chi_{c1}(1P)$ -21.3     -45.6    -44.1   -43.4   -35.7
                $\chi_{c2}(1P)$   NaN      5.28     4.39    4.99    4.01
                $\eta_{c}(2S)$    NaN -0.000348      NaN     NaN     nan
$\chi_{c0}(2P)$ $\psi_{1}(1D)$    NaN      -112     -119    -134    -212
$\chi_{c1}(2P)$ $\psi_{1}(1D)$    NaN     -28.9    -32.1   -59.1    -112
$\chi_{c2}(2P)$ $\psi_{1}(1D)$    NaN     0.924     1.21   0.827    2.59
$\chi_{c0}(3P)$ $\psi_{1}(1D)$    NaN  -0.00781   -0.452 -0.0132  -0.238
$\chi_{c1}(3P)$ $\psi_{1}(1D)$    NaN    -0.223   -0.022  -0.204  -0.141
$\chi_{c2}(3P)$ $\psi_{1}(1D)$    NaN   -0.0533 -0.00277 -0.0454 -0.0288

In [60]:
raise KeyboardInterrupt("Enough for output")

KeyboardInterrupt: Enough for output